#Book_Recargas

##configurações iniciais

In [1]:
!pip install spark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.7/351.7 kB 12.5 MB/s eta 0:00:00


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format, col, count, round, cast, desc,to_timestamp, concat_ws, substring, lit, lpad, countDistinct, row_number, isnan, when, corr, first, max as Fmax, min as Fmin, sum as Fsum, to_timestamp, to_date, concat
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql.window import Window

import builtins

In [3]:
spark = (
    SparkSession.builder
    .appName("book_recarga")

    # =========================
    # PARALELISMO
    # =========================
    .config("spark.sql.shuffle.partitions", "120")
    .config("spark.default.parallelism", "16")

    # =========================
    # MEMÓRIA (RAM 50GB)
    # =========================
    .config("spark.driver.memory", "32g")
    .config("spark.executor.memory", "32g")
    .config("spark.memory.fraction", "0.85")
    .config("spark.memory.storageFraction", "0.25")

    # =========================
    # EXECUÇÃO
    # =========================
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.adaptive.skewJoin.enabled", "true")

    # =========================
    # JOINS
    # =========================
    .config("spark.sql.autoBroadcastJoinThreshold", "100m")

    # =========================
    # SERIALIZAÇÃO
    # =========================
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryoserializer.buffer.max", "512m")

    # =========================
    # CACHE / COLUMNAR
    # =========================
    .config("spark.sql.inMemoryColumnarStorage.compressed", "true")
    .config("spark.sql.inMemoryColumnarStorage.batchSize", "20000")

    # =========================
    # GC
    # =========================
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")
    .config("spark.driver.extraJavaOptions", "-XX:+UseG1GC")

    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

In [4]:
from pyspark.sql import functions as F

In [5]:
from pyspark.sql import types as T

In [6]:
from pyspark.sql.window import Window

##Importação da base de público válida

In [7]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
colab = True

if colab==True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


antes de iniciar o book_recargas, traremos a base score full com todos CPFs que iremos considerar, para cruzar com as recargas e reduzir registros que não interessam para o público alvo (base recarga tem 100 MM de registros e cai para xx MM após filtro).

In [8]:
df_base_score_full = spark.read.parquet("/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202410",
                                        "/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202411",
                                        "/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202412",
                                        "/content/drive/MyDrive/hackathon/projeto_hackathon/01_INGESTAO/base_score/stg/base_score_full/SAFRA=202501")

### Exploração | tratamentos | **filtros**

In [9]:
#renomear safra para safra_buerau
df_base_score_full = df_base_score_full.withColumnRenamed("SAFRA", "SAFRA_BUREAU")

In [10]:
# Contagem de CPFs duplicados
df_dup = (
    df_base_score_full.groupBy("NUM_CPF")
    .agg(count("*").alias("qtd"))
    .filter(col("qtd") > 1)
)

print(f"CPFs duplicados: {df_dup.count():,}")
df_dup.show()

CPFs duplicados: 93,279
+-----------+---+
|    NUM_CPF|qtd|
+-----------+---+
|ZZZWX77TXWN|  2|
|ZZZU9Y8YW79|  2|
|ZZXY7TYUNZN|  2|
|ZZXWTUXWUZZ|  2|
|ZZWWNTN8WTT|  2|
|ZZWTTUWT9ZZ|  2|
|ZZUX7XN9T7U|  2|
|ZZTXYY9UYZ8|  2|
|ZZT7YN9WY7W|  2|
|ZZNTYNW7YWZ|  2|
|ZZ7X979UZ99|  2|
|ZZ7YYWUNZX8|  2|
|ZZ7NUYY7U7T|  2|
|ZZ8YXZYXXYT|  2|
|ZZ8WNNUXNTT|  2|
|ZZ8U8T7T87W|  2|
|ZZ9Z9ZT7YT7|  2|
|ZZ9WWXX8ZZ7|  2|
|ZXZN97ZNZ78|  3|
|ZXZ77X97XXZ|  2|
+-----------+---+
only showing top 20 rows


In [11]:
df_base_score_full.groupBy("NUM_CPF").count().filter("count > 1").count()

93279

In [12]:
# print(f"Linhas totais: {df_base_score_full.count():,}")
# print(f"CPFs únicos: {df_base_score_full.select('NUM_CPF').distinct().count():,}")

Linhas totais: 2,559,448
CPFs únicos: 2,462,481


In [12]:
#transformação formato variáveis numéricas
df_base_score_full = (
df_base_score_full    .withColumn("NUM_CPF", col("NUM_CPF").cast("string"))
    .withColumn("SCORE_01", col("SCORE_01").cast("double"))
    .withColumn("SCORE_02", col("SCORE_02").cast("double"))
    .withColumn("FPD", col("FPD").cast("int"))
)

In [13]:
df_cpfs_validos = df_base_score_full.select("NUM_CPF").distinct()

### IMPORTAÇÃO BASE RECARGAS

In [14]:
path_rec = "/content/drive/MyDrive/hackathon/base_recarga/bases"
df_recarga = spark.read.parquet(path_rec)

In [16]:
# print("✅ Linhas:", df_recarga.count())
# print("✅ Colunas:", len(df_recarga.columns))
# df_recarga.printSchema()
# df_recarga.show(5)

✅ Linhas: 100213651
✅ Colunas: 24
root
 |-- NUM_CPF: string (nullable = true)
 |-- DW_NUM_NTC: string (nullable = true)
 |-- DAT_INSERCAO_CREDITO: string (nullable = true)
 |-- HOR_INSERCAO_CREDITO: string (nullable = true)
 |-- DW_NUM_CLIENTE: string (nullable = true)
 |-- COD_TECNOLOGIA_DW: string (nullable = true)
 |-- COD_CANAL_AQUISICAO: string (nullable = true)
 |-- COD_TIPO_CREDITO: string (nullable = true)
 |-- COD_PROMOCAO: string (nullable = true)
 |-- VAL_CREDITO_INSERIDO: string (nullable = true)
 |-- VAL_BONUS: string (nullable = true)
 |-- VAL_REAL: string (nullable = true)
 |-- COD_PLATAFORMA_ATU: string (nullable = true)
 |-- COD_STATUS_PLATAFORMA: string (nullable = true)
 |-- IND_METODO_PAGAMENTO: string (nullable = true)
 |-- DW_PLANO_TARIFACAO: string (nullable = true)
 |-- DW_TIPO_RECARGA: string (nullable = true)
 |-- DW_TIPO_INSERCAO: string (nullable = true)
 |-- DW_FORMA_PAGAMENTO: string (nullable = true)
 |-- DW_INSTITUICAO: string (nullable = true)
 |-- COD_

###JOIN BASES

In [15]:
#join para usar somente os CPFs válidos
df_recarga_alvo = df_recarga.join(df_cpfs_validos, "NUM_CPF", "inner")

In [18]:
# print("✅ Linhas:", df_recarga_alvo.count())
# print("✅ Colunas:", len(df_recarga_alvo.columns))
# df_recarga_alvo.printSchema()
# df_recarga_alvo.show(5)

✅ Linhas: 69727469
✅ Colunas: 24
root
 |-- NUM_CPF: string (nullable = true)
 |-- DW_NUM_NTC: string (nullable = true)
 |-- DAT_INSERCAO_CREDITO: string (nullable = true)
 |-- HOR_INSERCAO_CREDITO: string (nullable = true)
 |-- DW_NUM_CLIENTE: string (nullable = true)
 |-- COD_TECNOLOGIA_DW: string (nullable = true)
 |-- COD_CANAL_AQUISICAO: string (nullable = true)
 |-- COD_TIPO_CREDITO: string (nullable = true)
 |-- COD_PROMOCAO: string (nullable = true)
 |-- VAL_CREDITO_INSERIDO: string (nullable = true)
 |-- VAL_BONUS: string (nullable = true)
 |-- VAL_REAL: string (nullable = true)
 |-- COD_PLATAFORMA_ATU: string (nullable = true)
 |-- COD_STATUS_PLATAFORMA: string (nullable = true)
 |-- IND_METODO_PAGAMENTO: string (nullable = true)
 |-- DW_PLANO_TARIFACAO: string (nullable = true)
 |-- DW_TIPO_RECARGA: string (nullable = true)
 |-- DW_TIPO_INSERCAO: string (nullable = true)
 |-- DW_FORMA_PAGAMENTO: string (nullable = true)
 |-- DW_INSTITUICAO: string (nullable = true)
 |-- COD_G

In [19]:
#quantidade de CPFs correspondidos entre bureau e recarga
# df_recarga_alvo.select("NUM_CPF").distinct().count()

In [16]:
df_00=df_recarga_alvo

###DW_NUM_CLIENTE

In [21]:
# df_00.groupBy("NUM_CPF").agg(F.countDistinct("DW_NUM_CLIENTE").alias("count_distinct_dw_num_cliente")).filter(F.col("count_distinct_dw_num_cliente") == 4).orderBy(F.col("count_distinct_dw_num_cliente").desc()).show()

In [22]:
# print(f"Total de DW_NUM_CLIENTE distintos na base: {df_00.select('DW_NUM_CLIENTE').distinct().count():,}")

retirar do book pois apresenta cardinalidade superior a 4MM!!

In [23]:
#df_00.filter(F.col('NUM_CPF') == '7777WYTZ9UU').orderBy('DW_NUM_CLIENTE', 'DAT_INSERCAO_CREDITO', 'HOR_INSERCAO_CREDITO').show(200)

### SAFRA_RECARGA

In [17]:
#períodos das recargas
from pyspark.sql.functions import to_date, min as Fmin, max as Fmax

(df_00
    .withColumn("DAT_INSERCAO_CREDITO_DT", to_date("DAT_INSERCAO_CREDITO", "ddMMMyyyy:HH:mm:ss"))
    .select(Fmin("DAT_INSERCAO_CREDITO_DT").alias("min_date"), Fmax("DAT_INSERCAO_CREDITO_DT").alias("max_date"))
    .show()
)

+----------+----------+
|  min_date|  max_date|
+----------+----------+
|2023-10-01|2025-03-31|
+----------+----------+



Transformações

In [18]:
#transformação da variável de inserção em campo data para depois construir variável de safra de recarga
df_00 = df_00.withColumn("DAT_INSERCAO_CREDITO_DT", to_date("DAT_INSERCAO_CREDITO", "ddMMMyyyy:HH:mm:ss"))
df_00 = df_00.withColumn("SAFRA_RECARGA", F.date_format(F.col("DAT_INSERCAO_CREDITO_DT"), "yyyyMM"))

In [19]:
df_00 = df_00.filter(~F.col("SAFRA_RECARGA").isin(["202502", "202503"]))
df_00.printSchema()

root
 |-- NUM_CPF: string (nullable = true)
 |-- DW_NUM_NTC: string (nullable = true)
 |-- DAT_INSERCAO_CREDITO: string (nullable = true)
 |-- HOR_INSERCAO_CREDITO: string (nullable = true)
 |-- DW_NUM_CLIENTE: string (nullable = true)
 |-- COD_TECNOLOGIA_DW: string (nullable = true)
 |-- COD_CANAL_AQUISICAO: string (nullable = true)
 |-- COD_TIPO_CREDITO: string (nullable = true)
 |-- COD_PROMOCAO: string (nullable = true)
 |-- VAL_CREDITO_INSERIDO: string (nullable = true)
 |-- VAL_BONUS: string (nullable = true)
 |-- VAL_REAL: string (nullable = true)
 |-- COD_PLATAFORMA_ATU: string (nullable = true)
 |-- COD_STATUS_PLATAFORMA: string (nullable = true)
 |-- IND_METODO_PAGAMENTO: string (nullable = true)
 |-- DW_PLANO_TARIFACAO: string (nullable = true)
 |-- DW_TIPO_RECARGA: string (nullable = true)
 |-- DW_TIPO_INSERCAO: string (nullable = true)
 |-- DW_FORMA_PAGAMENTO: string (nullable = true)
 |-- DW_INSTITUICAO: string (nullable = true)
 |-- COD_GRUPO_CARTAO: string (nullable = t

In [ ]:
# print(f"Linhas totais: {df_00.count():,}")
# print(f"CPFs únicos: {df_00.select('NUM_CPF').distinct().count():,}")

###Verificação dos nulos

In [ ]:
# #verificação dos nulos nas colunas
# from pyspark.sql.functions import col, sum as Fsum

# df_00.select([
#     Fsum(col(c).isNull().cast("int")).alias(c) for c in df_00.columns
# ]).show()

Atenção: Colunas que mais apresentaram nulos foram COD_PROMOCAO, DW_PLANO_TARIFACAO, DW_TIPO_RECARGA e VALOR_SOS.

### formatação de variáveis numéricas

In [21]:
#transformação das variáveis numéricas
df_00 = df_00 \
    .withColumn("VAL_CREDITO_INSERIDO", F.col("VAL_CREDITO_INSERIDO").cast("double")) \
    .withColumn("VAL_BONUS", F.col("VAL_BONUS").cast("double")) \
    .withColumn("VAL_REAL", F.col("VAL_REAL").cast("double")) \
    .withColumn("FLAG_SOS", F.col("FLAG_SOS").cast("double")) \
    .withColumn("VALOR_SOS", F.col("VALOR_SOS").cast("double"))


### Hora recarga

In [22]:
df_00 = df_00.withColumn("HOR_INSERCAO_CREDITO2", F.lpad(F.col("HOR_INSERCAO_CREDITO"), 6, "0"))
df_00.select("HOR_INSERCAO_CREDITO", "HOR_INSERCAO_CREDITO2").show(10)




+--------------------+---------------------+
|HOR_INSERCAO_CREDITO|HOR_INSERCAO_CREDITO2|
+--------------------+---------------------+
|              104621|               104621|
|              200602|               200602|
|               53935|               053935|
|              120738|               120738|
|               74300|               074300|
|               93822|               093822|
|              100254|               100254|
|               80406|               080406|
|              234702|               234702|
|               50145|               050145|
+--------------------+---------------------+
only showing top 10 rows


In [23]:
df_00 = df_00.withColumn(
    "HOR_INSERCAO_CREDITO_TIME",
    to_timestamp(
        concat(lit("1970-01-01 "), substring(col("HOR_INSERCAO_CREDITO2"), 1, 2), lit(":"),
               substring(col("HOR_INSERCAO_CREDITO2"), 3, 2), lit(":"),
               substring(col("HOR_INSERCAO_CREDITO2"), 5, 2)),
        "yyyy-MM-dd HH:mm:ss"
    )
)


In [24]:
df_00.select("HOR_INSERCAO_CREDITO", "HOR_INSERCAO_CREDITO2", "HOR_INSERCAO_CREDITO_TIME").show(10, truncate=False)

df_00 = df_00.withColumn(
    "HOR_INSERCAO_CREDITO_ONLY_TIME",
    date_format(col("HOR_INSERCAO_CREDITO_TIME"), "HH:mm:ss")
)

df_00.select("HOR_INSERCAO_CREDITO", "HOR_INSERCAO_CREDITO2", "HOR_INSERCAO_CREDITO_ONLY_TIME").show(10, truncate=False)

df_00.printSchema()

+--------------------+---------------------+-------------------------+
|HOR_INSERCAO_CREDITO|HOR_INSERCAO_CREDITO2|HOR_INSERCAO_CREDITO_TIME|
+--------------------+---------------------+-------------------------+
|104621              |104621               |1970-01-01 10:46:21      |
|200602              |200602               |1970-01-01 20:06:02      |
|53935               |053935               |1970-01-01 05:39:35      |
|120738              |120738               |1970-01-01 12:07:38      |
|74300               |074300               |1970-01-01 07:43:00      |
|93822               |093822               |1970-01-01 09:38:22      |
|100254              |100254               |1970-01-01 10:02:54      |
|80406               |080406               |1970-01-01 08:04:06      |
|234702              |234702               |1970-01-01 23:47:02      |
|50145               |050145               |1970-01-01 05:01:45      |
+--------------------+---------------------+-------------------------+
only s

In [25]:
# #filtro para validações
# df_00.filter(df_00['NUM_CPF'] == '7ZT78ZZXX87').orderBy('SAFRA_RECARGA').show()

In [26]:
#transforma coluna hora recarga em int de 0-23
df_00 = df_00.withColumn("HORA_RECARGA", F.substring(F.col("HOR_INSERCAO_CREDITO_ONLY_TIME"), 1, 2).cast(T.IntegerType()))

df_00.select("HOR_INSERCAO_CREDITO_ONLY_TIME", "HORA_RECARGA").show(5)

+------------------------------+------------+
|HOR_INSERCAO_CREDITO_ONLY_TIME|HORA_RECARGA|
+------------------------------+------------+
|                      10:46:21|          10|
|                      20:06:02|          20|
|                      05:39:35|           5|
|                      12:07:38|          12|
|                      07:43:00|           7|
+------------------------------+------------+
only showing top 5 rows


In [27]:
df_hora_recarga = (
    df_00.groupBy("HORA_RECARGA")
    .agg(count("NUM_CPF").alias("qtd"))

)
df_hora_recarga.sort("qtd").orderBy("HORA_RECARGA").show(24)

+------------+-------+
|HORA_RECARGA|    qtd|
+------------+-------+
|           0|3946339|
|           1|2887593|
|           2|2610027|
|           3|2409172|
|           4|1148611|
|           5|2681795|
|           6|1759864|
|           7|2633226|
|           8|2988751|
|           9|3341977|
|          10|3386022|
|          11|3310571|
|          12|3259160|
|          13|2993948|
|          14|2776863|
|          15|2661228|
|          16|2631871|
|          17|2711688|
|          18|2746752|
|          19|2514408|
|          20|2083592|
|          21|1638879|
|          22|1243489|
|          23|1754956|
+------------+-------+



In [28]:
#Variáveis derivadas de hora da recarga
df = df_00.withColumn(
    "fx_hora",
    F.when(F.col("HORA_RECARGA").between(0, 5),  "01_Madrugada")
     .when(F.col("HORA_RECARGA").between(6, 11), "02_Manha")
     .when(F.col("HORA_RECARGA").between(12,17), "03_Tarde")
     .otherwise("04_Noite")
)

In [29]:
#contagem por safra
df_fx_hora = (
    df
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.sum(F.when(F.col("fx_hora") == "01_Madrugada", 1).otherwise(0)).alias("qtd_rec_madrugada"),
        F.sum(F.when(F.col("fx_hora") == "02_Manha", 1).otherwise(0)).alias("qtd_rec_manha"),
        F.sum(F.when(F.col("fx_hora") == "03_Tarde", 1).otherwise(0)).alias("qtd_rec_tarde"),
        F.sum(F.when(F.col("fx_hora") == "04_Noite", 1).otherwise(0)).alias("qtd_rec_noite")
    )
)

In [30]:
#horário dominante
df_counts = df.groupBy("NUM_CPF", "SAFRA_RECARGA", "fx_hora").count()

w = Window.partitionBy("NUM_CPF", "SAFRA_RECARGA").orderBy(F.col("count").desc())

df_hora_dom = (
    df_counts
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .select(
        "NUM_CPF",
        "SAFRA_RECARGA",
        F.col("fx_hora").alias("fx_hora_dominante")
    )
)

### criação de variáveis a partir das variáveis numéricas

In [31]:
df_num = (
    df
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.count("*").alias("qtd_recargas"),
        F.sum("VAL_CREDITO_INSERIDO").alias("vlr_total_recarga"),
        F.avg("VAL_CREDITO_INSERIDO").alias("vlr_medio_recarga"),
        F.max("VAL_CREDITO_INSERIDO").alias("vlr_max_recarga"),
        F.min("VAL_CREDITO_INSERIDO").alias("vlr_min_recarga"),
        F.countDistinct("DW_NUM_NTC").alias("qtd_telefones"),
        F.min("DAT_INSERCAO_CREDITO_DT").alias("dt_primeira_recarga"),
        F.max("DAT_INSERCAO_CREDITO_DT").alias("dt_ultima_recarga"),
        F.sum("VAL_BONUS").alias("vlr_total_bonus"),
        F.avg("VAL_BONUS").alias("vlr_medio_bonus"),
        F.sum("VAL_REAL").alias("vlr_total_real"),
        F.avg("VAL_REAL").alias("vlr_medio_real")
    )
)

### SOS

In [32]:
#variáveis derivadas de SOS
df_sos = (
    df
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.sum("FLAG_SOS").alias("qtd_rec_sos"),
        F.count("*").alias("qtd_rec_total"),
        F.avg(F.when(F.col("FLAG_SOS") == 1, F.col("VALOR_SOS"))).alias("vlr_medio_rec_sos")
    )
    .withColumn("pct_rec_sos", F.col("qtd_rec_sos") / F.col("qtd_rec_total"))
    .withColumn("flg_uso_sos", F.when(F.col("qtd_rec_sos") > 0, 1).otherwise(0))
    .drop("qtd_rec_total")
)

### Bonus

In [33]:
#variáveis derivadas de bônus de recarga
df_bonus = (
    df
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.sum("VAL_BONUS").alias("vlr_total_bonus"),
        F.avg("VAL_BONUS").alias("vlr_medio_bonus"),
        F.count(F.when(F.col("VAL_BONUS") > 0, True)).alias("qtd_rec_com_bonus"),
        F.count("*").alias("qtd_rec_total"),
        F.sum(F.when(F.col("VAL_BONUS") > 0, F.col("VAL_REAL")).otherwise(0)).alias("vlr_rec_com_bonus")
    )
    .withColumn("pct_rec_com_bonus", F.col("qtd_rec_com_bonus") / F.col("qtd_rec_total"))
    .withColumn(
        "fx_dependencia_bonus",
        F.when(F.col("pct_rec_com_bonus") == 0, "01_Nao_Usa")
         .when(F.col("pct_rec_com_bonus") <= 0.3, "02_Baixa")
         .when(F.col("pct_rec_com_bonus") <= 0.7, "03_Media")
         .otherwise("04_Alta")
    )
    .drop("qtd_rec_total")
)

### Canal aquisição

In [34]:
#variáveis cat - ex. canal dominante
df_canal_counts = (
    df
    .groupBy("NUM_CPF", "SAFRA_RECARGA", "COD_CANAL_AQUISICAO")
    .count()
)

w = Window.partitionBy("NUM_CPF", "SAFRA_RECARGA").orderBy(F.col("count").desc())

df_canal = (
    df_canal_counts
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .select(
        "NUM_CPF",
        "SAFRA_RECARGA",
        F.col("COD_CANAL_AQUISICAO").alias("canal_principal")
    )
)

### Variáveis relacionadas a datas (recencia, gaps)

cálculo de recência - diferença entre a data de referência da safra e a última recarga do CPF

Importante: NÃO usar current_date() em book histórico

Usar data fim da safra ou data de corte.

In [35]:
#data de referencia da safra
df_safra_ref = spark.createDataFrame([
    ("202310", "2023-10-31"),
    ("202311", "2023-11-30"),
    ("202312", "2023-12-31"),
    ("202401", "2024-01-31"),
    ("202402", "2024-02-29"),
    ("202403", "2024-03-31"),
    ("202404", "2024-04-30"),
    ("202405", "2024-05-31"),
    ("202406", "2024-06-30"),
    ("202407", "2024-07-31"),
    ("202408", "2024-08-31"),
    ("202409", "2024-09-30"),
    ("202410", "2024-10-31"),
    ("202411", "2024-11-30"),
    ("202412", "2024-12-31"),
    ("202501", "2025-01-31"),
], ["SAFRA_RECARGA", "DT_REF"])

df_safra_ref = df_safra_ref.withColumn("dt_ref", F.to_date("dt_ref"))

In [36]:
#data da última recarga já calculamos no df_book

In [37]:
df_num2 = (
    df
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(

        F.max("DAT_INSERCAO_CREDITO_DT").alias("dt_ultima_recarga"),
    )
)

In [38]:
#calculo da recencia
df_recencia = (
    df_num2
    .join(df_safra_ref, "SAFRA_RECARGA", "left")
    .withColumn(
        "recencia_dias",
        F.datediff(F.col("dt_ref"), F.col("dt_ultima_recarga"))
    )
    .drop("dt_ref", "dt_ultima_recarga")
)

Resultado:

0 → recarregou no último dia da safra

se >30 → cliente frio

null → não recarregou na safra

In [39]:
df_recencia = df_recencia.withColumn(
    "fx_recencia",
    F.when(F.col("recencia_dias") <= 7, "01_0a7")
     .when(F.col("recencia_dias") <= 15, "02_8a15")
     .when(F.col("recencia_dias") <= 30, "03_16a30")
     .when(F.col("recencia_dias") <= 60, "04_31a60")
     .otherwise("05_60+")
)

In [40]:
#indicadores comportamentais

w = Window.partitionBy("NUM_CPF", "SAFRA_RECARGA").orderBy("DAT_INSERCAO_CREDITO_DT")

df_gap = (
    df
    .withColumn("dt_rec_anterior", F.lag("DAT_INSERCAO_CREDITO_DT").over(w))
    .withColumn("gap_dias", F.datediff("DAT_INSERCAO_CREDITO_DT", "dt_rec_anterior"))
)

df_gap_book = (
    df_gap
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.avg("gap_dias").alias("gap_medio_recarga"),
        F.stddev("gap_dias").alias("gap_std_recarga")
    )
)



recencia média entre recargas - mede a cadência natural do cliente
📌 Interpretação:

Gap baixo → cliente recorrente

Gap alto → cliente oportunista

In [41]:
#Desvio da recência vs padrão do cliente
#Sinal fortíssimo de churn precoce.
df_recencia_enriq = (
    df_recencia
    .join(df_gap_book, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .withColumn(
        "desvio_recencia",
        F.col("recencia_dias") - F.col("gap_medio_recarga")
    )
)

In [42]:
#Indicadores de risco / churn - flag de quebra de padrão - Melhor que regra fixa de 30 dias.
df_recencia_enriq = df_recencia_enriq.withColumn(
    "flg_quebra_padrao",
    F.when(F.col("recencia_dias") > 1.5 * F.col("gap_medio_recarga"), 1).otherwise(0)
)

In [43]:
#score simples de risco (heurístico)
df_recencia_enriq = df_recencia_enriq.withColumn(
    "score_risco_recencia",
    F.when(F.col("recencia_dias") <= 7, 0)
     .when(F.col("recencia_dias") <= 15, 1)
     .when(F.col("recencia_dias") <= 30, 2)
     .otherwise(3)
)

### COD_PROMOCAO

In [44]:
# df_00.select(F.countDistinct("COD_PROMOCAO")).show()

In [45]:
# df_00.groupBy("COD_PROMOCAO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

###COD_TECNOLOGIA_DW

In [46]:
# df_00.select(F.countDistinct("COD_TECNOLOGIA_DW")).show()

In [47]:
# df_00.groupBy("COD_TECNOLOGIA_DW").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

### DW_PLANO_TARIFACAO

In [48]:
# df_00.select(F.countDistinct("DW_PLANO_TARIFACAO")).show()

In [49]:
# df_00.groupBy("DW_PLANO_TARIFACAO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

In [50]:
df_00 = df_00.withColumn(
    "PLANO_TARIFACAO",
    F.when(F.col("DW_PLANO_TARIFACAO") == "541200", "541200")
     .when(F.col("DW_PLANO_TARIFACAO") == "668900", "668900")
     .when(F.col("DW_PLANO_TARIFACAO") == "599100", "599100")
     .when(F.col("DW_PLANO_TARIFACAO") == "607690", "607690")
     .when(F.col("DW_PLANO_TARIFACAO") == "369889", "369889")
     .when(F.col("DW_PLANO_TARIFACAO") == "393401", "393401")
     .otherwise("999999")
)
#df_00.select("DW_PLANO_TARIFACAO", "PLANO_TARIFACAO").show(10, truncate=False)

In [51]:
# df_00.groupBy("PLANO_TARIFACAO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

In [52]:
#contagem por safra
df_plano = (
    df_00
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.count(F.when(F.col("PLANO_TARIFACAO") == "541200", 1).otherwise(0)).alias("qtd_rec_plano_541200"),
        F.count(F.when(F.col("PLANO_TARIFACAO") == "668900", 1).otherwise(0)).alias("qtd_rec_plano_668900"),
        F.count(F.when(F.col("PLANO_TARIFACAO") == "599100", 1).otherwise(0)).alias("qtd_rec_plano_599100"),
        F.count(F.when(F.col("PLANO_TARIFACAO") == "607690", 1).otherwise(0)).alias("qtd_rec_plano_607690"),
        F.count(F.when(F.col("PLANO_TARIFACAO") == "999999", 1).otherwise(0)).alias("qtd_rec_plano_999999"),
        F.count(F.when(F.col("PLANO_TARIFACAO") == "369889", 1).otherwise(0)).alias("qtd_rec_plano_369889"),
        F.count(F.when(F.col("PLANO_TARIFACAO") == "393401", 1).otherwise(0)).alias("qtd_rec_plano_393401")
    )
)

### DW_TIPO_RECARGA

In [53]:
# df_00.select(F.countDistinct("DW_TIPO_RECARGA")).show()

In [54]:
# df_00.groupBy("DW_TIPO_RECARGA").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

In [55]:
#criação de flag do tipo_recarga_adicional_controle
from pyspark.sql import functions as F

df_recargas = df_00.withColumn(
    "flg_recarga_adicional_controle",
    F.when(F.col("DW_TIPO_RECARGA") == 2, 1).otherwise(0)
)

In [56]:
df_feat = (
    df_recargas
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.sum("flg_recarga_adicional_controle").alias("qtd_recarga_adicional_controle"),
        F.count("*").alias("qtd_rec_total")
    )
)

In [57]:
df_feat = df_feat.withColumn(
    "flg_uso_recarga_adicional_controle",
    F.when(F.col("qtd_recarga_adicional_controle") > 0, 1).otherwise(0)
)

In [58]:
df_feat = df_feat.withColumn(
    "pct_recarga_adicional_controle",
    F.when(F.col("qtd_rec_total") > 0,
           F.col("qtd_recarga_adicional_controle") / F.col("qtd_rec_total"))
     .otherwise(0.0)
)
df_feat = df_feat.drop("qtd_rec_total")

| Feature                     | Significado                                |
| --------------------------- | ------------------------------------------ |
| `flg_uso_recarga_adicional` | Cliente fez ao menos uma recarga adicional |
| `qtd_recarga_adicional`     | Quantidade de recargas adicionais          |
| `pct_recarga_adicional`     | Intensidade do uso de adicional            |


A recarga adicional do Claro Controle é identificada exclusivamente pelo valor DW_TIPO_RECARGA = 2. Devido à ausência de preenchimento informativo para recargas padrão, o campo é tratado de forma binária, gerando indicadores de uso e intensidade de recarga adicional.

### COD_CANAL_AQUISICAO

In [59]:
# df_00.select(F.countDistinct("COD_CANAL_AQUISICAO")).show()

In [60]:
# df_00.groupBy("COD_CANAL_AQUISICAO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

### COD_TIPO_CREDITO

In [61]:
# df_00.select(F.countDistinct("COD_TIPO_CREDITO")).show()

In [62]:
# df_00.groupBy("COD_TIPO_CREDITO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

como o tipo de crédito é predominantemente online vamos retirar a variável!

---




### IND_METODO_PAGAMENTO

In [63]:
# df_00.select(F.countDistinct("IND_METODO_PAGAMENTO")).show()

In [64]:
# df_00.groupBy("IND_METODO_PAGAMENTO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

como o indicador de metodo de pagamento é predominantemente A vamos retirar a variável!

### DW_TIPO_INSERCAO

In [65]:
# df_00.select(F.countDistinct("DW_TIPO_INSERCAO")).show()

In [66]:
# df_00.groupBy("DW_TIPO_INSERCAO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

In [67]:
df_tipo_insercao = (
    df_00.groupBy("SAFRA_RECARGA", "NUM_CPF")
    .agg(F.countDistinct("DW_TIPO_INSERCAO").alias("qtd_tipo_insercao"))
)



### DW_FORMA_PAGAMENTO

In [68]:
# df_00.select(F.countDistinct("DW_FORMA_PAGAMENTO")).show()

In [69]:
# df_00.groupBy("DW_FORMA_PAGAMENTO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

In [70]:
df_forma_pagamento_counts = (
    df_00.groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "-1", 1)).alias("qtd_forma_pgto_na"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "-2", 1)).alias("qtd_forma_pgto_nd"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "-3", 1)).alias("qtd_forma_pgto_ni"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "cp", 1)).alias("qtd_forma_pgto_cp"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "dd", 1)).alias("qtd_forma_pgto_dd"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "mp", 1)).alias("qtd_forma_pgto_mp"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "pb", 1)).alias("qtd_forma_pgto_pb"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "ca", 1)).alias("qtd_forma_pgto_ca"),
        F.count(F.when(F.col("DW_FORMA_PAGAMENTO") == "pa", 1)).alias("qtd_forma_pgto_pa")
    )
)

### COD_STATUS_PLATAFORMA

In [71]:
# df_00.select(F.countDistinct("COD_STATUS_PLATAFORMA")).show()

In [72]:
# df_00.groupBy("COD_STATUS_PLATAFORMA").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

As variáveis de status de plataforma foram transformadas em indicadores de atividade, risco e ruído cadastral. O código original foi descartado devido à alta concentração no estado ativo, priorizando métricas proporcionais e flags de expiração recente.

In [73]:
df_status = (
    df_00
    # ---------------- Linha ativa ----------------
    .withColumn(
        "flg_linha_ativa",
        F.when(F.col("COD_STATUS_PLATAFORMA").isin("A", "ST"), 1).otherwise(0)
    )

    # ---------------- Expirado recente ----------------
    .withColumn(
        "flg_expirado_recente",
        F.when(F.col("COD_STATUS_PLATAFORMA") == "ZB1", 1).otherwise(0)
    )

    # ---------------- Churn profundo ----------------
    .withColumn(
        "flg_churn_profundo",
        F.when(F.col("COD_STATUS_PLATAFORMA").isin("ZB2", "C"), 1).otherwise(0)
    )

    # ---------------- Ruído cadastral ----------------
    .withColumn(
        "flg_status_ruidos",
        F.when(
            F.col("COD_STATUS_PLATAFORMA")
            .isin("-3", "U", "F", "V", "S", "AT"),
            1
        ).otherwise(0)
    )
)


In [74]:
df_status_feat = (
    df_status
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.count("*").alias("qtd_eventos_status"),

        F.sum("flg_linha_ativa").alias("qtd_linha_ativa"),
        F.sum("flg_expirado_recente").alias("qtd_expirado_recente"),
        F.sum("flg_churn_profundo").alias("qtd_churn_profundo"),
        F.max("flg_linha_ativa").alias("flg_linha_ativa"),
        F.max("flg_expirado_recente").alias("flg_expirado_recente"),
        F.max("flg_churn_profundo").alias("flg_churn_profundo"),
        F.max("flg_status_ruidos").alias("flg_status_ruido"),

        F.countDistinct("COD_STATUS_PLATAFORMA").alias("qtd_status_distintos")
    )
)


In [75]:
df_status_feat = (
    df_status_feat
    .withColumn(
        "pct_linha_ativa",
        F.col("qtd_linha_ativa") / F.col("qtd_eventos_status")
    )
    .withColumn(
        "pct_expirado_recente",
        F.col("qtd_expirado_recente") / F.col("qtd_eventos_status")
    )
)


In [76]:
df_status_feat = (
    df_status_feat
    .withColumn(
        "flg_cliente_elegivel",
        F.when(
            (F.col("pct_linha_ativa") > 0.5) &
            (F.col("qtd_churn_profundo") == 0),
            1
        ).otherwise(0)
    ).drop("qtd_eventos_status","qtd_linha_ativa","qtd_expirado_recente","qtd_churn_profundo")
)


### DW_INSTITUICAO

In [77]:
# df_00.groupBy("DW_INSTITUICAO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

In [78]:
from pyspark.sql import functions as F

df_dim_instituicao = (
    spark.read
    .option("header", True)
    .option("delimiter", ",")
    .csv("/content/drive/MyDrive/hackathon/base_recarga/BI_DIM_INSTITUICAO.csv")
)

In [79]:
df_dim_instituicao.show(5)

+--------------+---------------+--------------------+--------------------+--------------------+--------------+----------------+------------------+----------+
|DW_INSTITUICAO|COD_INSTITUICAO|     DSC_INSTITUICAO|COD_TIPO_INSTITUICAO|DSC_TIPO_INSTITUICAO|COD_SISTEMA_DW|DAT_EXPIRACAO_DW|    DAT_CRIACAO_DW|COD_AGENTE|
+--------------+---------------+--------------------+--------------------+--------------------+--------------+----------------+------------------+----------+
|         13926|            416|         Banco Inter|                   1|               Banco|            21|            NULL|26AUG2021:00:18:55|      5416|
|         13927|            490|    MWallet/Bradesco|                   1|               Banco|            21|            NULL|26AUG2021:00:18:55|      5490|
|         13928|            514|Santander/C.Bancario|                   1|               Banco|            21|            NULL|26AUG2021:00:18:55|      5514|
|         13929|             24|      CEF/C.Bancario

In [80]:
#NORMALIZA
df_dim_inst_norm = (
    df_dim_instituicao
        .withColumn(
            "DSC_TIPO_INST_NORM",
            F.upper(F.trim(F.col("DSC_TIPO_INSTITUICAO")))
        )
)


In [81]:
df_dim_inst_cat = (
    df_dim_inst_norm
        .withColumn(
            "CATEGORIA_TIPO_INSTITUICAO",
            F.when(F.col("DSC_TIPO_INSTITUICAO").isin("NA", "ND", "NI"), "TECNICO")

            .when(
                F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("CARTEIRA")
                | F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("WALLET")
                | F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("DIGITAL"),
                "DIGITAL_WALLET"
            )

            .when(
                F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("BANCO")
                | F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("FINANCEIRA"),
                "BANCO"
            )

            .when(
                F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("VAREJO")
                | F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("LOJA"),
                "VAREJO_FISICO"
            )

            .when(
                F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("OPERADORA")
                | F.upper(F.col("DSC_TIPO_INSTITUICAO")).contains("TELECOM"),
                "TELECOM_CANAL"
            )

            .otherwise("OUTROS")
        )
)


In [82]:
df_dim_inst_cat.show(10)

+--------------+---------------+--------------------+--------------------+--------------------+--------------+----------------+------------------+----------+------------------+--------------------------+
|DW_INSTITUICAO|COD_INSTITUICAO|     DSC_INSTITUICAO|COD_TIPO_INSTITUICAO|DSC_TIPO_INSTITUICAO|COD_SISTEMA_DW|DAT_EXPIRACAO_DW|    DAT_CRIACAO_DW|COD_AGENTE|DSC_TIPO_INST_NORM|CATEGORIA_TIPO_INSTITUICAO|
+--------------+---------------+--------------------+--------------------+--------------------+--------------+----------------+------------------+----------+------------------+--------------------------+
|         13926|            416|         Banco Inter|                   1|               Banco|            21|            NULL|26AUG2021:00:18:55|      5416|             BANCO|                     BANCO|
|         13927|            490|    MWallet/Bradesco|                   1|               Banco|            21|            NULL|26AUG2021:00:18:55|      5490|             BANCO|        

In [83]:
#join com base de recargas
df_dim_inst_cat_selected = df_dim_inst_cat.select(
    "DW_INSTITUICAO",
    "COD_INSTITUICAO",
    "CATEGORIA_TIPO_INSTITUICAO"
)

df_rec_instituicao = (
    df_00
        .join(
            df_dim_inst_cat_selected,
            df_00.DW_INSTITUICAO == df_dim_inst_cat_selected.DW_INSTITUICAO,
            "left"
        )
        .withColumn(
            "CATEGORIA_TIPO_INSTITUICAO",
            F.coalesce(F.col("CATEGORIA_TIPO_INSTITUICAO"), F.lit("OUTROS"))
        )
)

In [84]:
#Variáveis categóricas
df_inst = (
    df_rec_instituicao
        # Flag banco
        .withColumn(
            "FLG_RECARGA_BANCO",
            F.when(F.col("CATEGORIA_TIPO_INSTITUICAO") == "BANCO", 1).otherwise(0)
        )

        # Flag digital
        .withColumn(
            "FLG_RECARGA_DIGITAL",
            F.when(F.col("CATEGORIA_TIPO_INSTITUICAO") == "DIGITAL_WALLET", 1).otherwise(0)
        )

        # Flag varejo físico
        .withColumn(
            "FLG_RECARGA_VAREJO",
            F.when(F.col("CATEGORIA_TIPO_INSTITUICAO") == "VAREJO_FISICO", 1).otherwise(0)
        )

        # Flag telecom
        .withColumn(
            "FLG_RECARGA_TELECOM",
            F.when(F.col("CATEGORIA_TIPO_INSTITUICAO") == "TELECOM_CANAL", 1).otherwise(0)
        )

        # Flag técnico / fallback
        .withColumn(
            "FLG_RECARGA_TECNICA",
            F.when(F.col("CATEGORIA_TIPO_INSTITUICAO") == "TECNICO", 1).otherwise(0)
        )
)


In [85]:
#agregacao num_cpf + safra
df_instituicao = (
    df_inst
        .groupBy("NUM_CPF", "SAFRA_RECARGA")
        .agg(
            F.count("*").alias("QTD_RECARGAS_TOTAL"),
            F.sum("FLG_RECARGA_DIGITAL").alias("QTD_RECARGAS_DIGITAL"),
            F.sum("FLG_RECARGA_BANCO").alias("QTD_RECARGAS_BANCO"),
            F.sum("FLG_RECARGA_VAREJO").alias("QTD_RECARGAS_VAREJO"),
            F.sum("FLG_RECARGA_TELECOM").alias("QTD_RECARGAS_TELECOM"),
            Fmax("FLG_RECARGA_BANCO").alias("FLG_RECARGA_BANCO"),
            Fmax("FLG_RECARGA_DIGITAL").alias("FLG_RECARGA_DIGITAL"),
            Fmax("FLG_RECARGA_VAREJO").alias("FLG_RECARGA_VAREJO"),
            Fmax("FLG_RECARGA_TELECOM").alias("FLG_RECARGA_TELECOM"),
            Fmax("FLG_RECARGA_TECNICA").alias("FLG_RECARGA_TECNICA")


        )
)


In [86]:
df_instituicao = (
    df_instituicao
        .withColumn(
            "PCT_RECARGA_DIGITAL",
            F.col("QTD_RECARGAS_DIGITAL") / F.col("QTD_RECARGAS_TOTAL")
        )
        .withColumn(
            "PCT_RECARGA_BANCO",
            F.col("QTD_RECARGAS_BANCO") / F.col("QTD_RECARGAS_TOTAL")
        )
        .withColumn(
            "PCT_RECARGA_VAREJO",
            F.col("QTD_RECARGAS_VAREJO") / F.col("QTD_RECARGAS_TOTAL")
        )
        .withColumn(
            "PCT_RECARGA_TELECOM",
            F.col("QTD_RECARGAS_TELECOM") / F.col("QTD_RECARGAS_TOTAL")
        )
)

In [87]:
df_instituicao = (
    df_instituicao
        .withColumn(
            "INSTITUICAO_DOMINANTE_RECARGA",
            F.when(
                F.col("QTD_RECARGAS_DIGITAL") >= F.greatest(
                    "QTD_RECARGAS_BANCO",
                    "QTD_RECARGAS_VAREJO",
                    "QTD_RECARGAS_TELECOM"
                ),
                "DIGITAL"
            ).when(
                F.col("QTD_RECARGAS_BANCO") >= F.greatest(
                    "QTD_RECARGAS_VAREJO",
                    "QTD_RECARGAS_TELECOM"
                ),
                "BANCO"
            ).when(
                F.col("QTD_RECARGAS_VAREJO") >= F.col("QTD_RECARGAS_TELECOM"),
                "VAREJO"
            ).otherwise("TELECOM")
        )
)


In [88]:
df_instituicao_fim = (
    df_instituicao
        .dropDuplicates(["NUM_CPF", "SAFRA_RECARGA"])
)


In [89]:
# df_instituicao_fim.show(10)

In [90]:
# df_instituicao_fim.groupBy("NUM_CPF", "SAFRA_RECARGA").count().filter("count > 1").count()


### COD_GRUPO_CARTAO

In [91]:
# df_00.groupBy("COD_GRUPO_CARTAO").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

In [92]:
#Contagem de recargas por grupo de cartão
df_grp_cartao = (
    df_00
    .groupBy("NUM_CPF", "SAFRA_RECARGA", "COD_GRUPO_CARTAO")
    .agg(F.count("*").alias("qtd_recargas_grupo"))
)


In [93]:
#Total de recargas por CPF + Safra
df_grp_cartao2 = (
    df_grp_cartao
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(F.sum("qtd_recargas_grupo").alias("qtd_recargas_total"))
)


In [94]:
#percentual
df_grp_cartao = (
    df_grp_cartao
    .join(df_grp_cartao2, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .withColumn(
        "pct_grupo_cartao",
        F.col("qtd_recargas_grupo") / F.col("qtd_recargas_total")
    )
)


In [95]:
w_dominante = (
    Window
    .partitionBy("NUM_CPF", "SAFRA_RECARGA")
    .orderBy(F.desc("qtd_recargas_grupo"))
)

df_dom = (
    df_grp_cartao
    .withColumn("rn", F.row_number().over(w_dominante))
    .filter(F.col("rn") == 1)
    .select(
        "NUM_CPF",
        "SAFRA_RECARGA",
        F.col("COD_GRUPO_CARTAO").alias("GRUPO_CARTAO_DOMINANTE"),
        F.col("pct_grupo_cartao").alias("PCT_GRUPO_CARTAO_DOMINANTE")
    )
)


In [96]:
df_qtd_distintos = (
    df_grp_cartao
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.countDistinct("COD_GRUPO_CARTAO")
        .alias("QTD_GRUPOS_CARTAO_DISTINTOS")
    )
)


In [97]:
df_features_grupo_cartao = (
    df_dom
    .join(df_qtd_distintos, ["NUM_CPF", "SAFRA_RECARGA"], "left")
)


| Coluna                        | Descrição                       |
| ----------------------------- | ------------------------------- |
| `NUM_CPF`                     | Identificador do cliente        |
| `SAFRA_RECARGA`               | Safra temporal                  |
| `GRUPO_CARTAO_DOMINANTE`      | Grupo mais utilizado            |
| `PCT_GRUPO_CARTAO_DOMINANTE`  | Concentração no grupo dominante |
| `QTD_GRUPOS_CARTAO_DISTINTOS` | Diversidade de grupos           |


### DSC_GRUPO_CARTAO_WPP

In [98]:
# df_00.groupBy("DSC_GRUPO_CARTAO_WPP").agg(F.count("NUM_CPF").alias("count_cpf")).orderBy(F.col("count_cpf").desc()).show(truncate=False)

| Grupo            | Leitura funcional                             |
| ---------------- | --------------------------------------------- |
| **NaoSeAplica**  | Recarga padrão, sem vínculo com pacote/cartão |
| **Rec.Online**   | Recarga avulsa digital                        |
| **AtivPromocao** | Ativação de promoção                          |
| **ChipPre+R$XX** | Compra/ativação de chip com crédito           |
| **ForcaZB2**     | Ação comercial / força de vendas              |
| **Pl. Controle** | Plano Controle                                |
| **CartaoFisico** | Recarga via cartão físico                     |
| **PCT …**        | Pacotes específicos (voz, SMS, LDN etc.)      |
| **Rec.Teste**    | Recarga de teste                              |
| **-2**           | Não determinado (ruído)                       |


In [99]:
#Normalização do texto
df_rec_grupo_cartao_wpp = (
    df_00
    .withColumn(
        "DSC_GRUPO_CARTAO_WPP_NORM",
        F.lower(F.col("DSC_GRUPO_CARTAO_WPP"))
    )
)


In [100]:
#Criação da categoria semântica
df_rec_grupo_cartao_wpp = (
    df_rec_grupo_cartao_wpp
    .withColumn(
        "CAT_DSC_GRUPO_CARTAO",
        F.when(F.col("DSC_GRUPO_CARTAO_WPP_NORM").rlike("rec\\.online|recarga"), "RECARGA_ONLINE")
         .when(F.col("DSC_GRUPO_CARTAO_WPP_NORM").rlike("ativ|promo"), "ATIVACAO_PROMO")
         .when(F.col("DSC_GRUPO_CARTAO_WPP_NORM").rlike("cartao"), "CARTAO_FISICO")
         .when(F.col("DSC_GRUPO_CARTAO_WPP_NORM").rlike("pct|pacote"), "PACOTE_SERVICO")
         .when(F.col("DSC_GRUPO_CARTAO_WPP_NORM").rlike("chip"), "CHIP")
         .otherwise("OUTROS")
    )
)


In [101]:
#Contagem por CPF + Safra + Categoria
df_cat = (
    df_rec_grupo_cartao_wpp
    .groupBy("NUM_CPF", "SAFRA_RECARGA", "CAT_DSC_GRUPO_CARTAO")
    .agg(F.count("*").alias("qtd_rec"))
)



In [102]:
#Total de recargas por CPF + Safra
df_total_cat = (
    df_cat
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(F.sum("qtd_rec").alias("qtd_rec_total"))
)

In [103]:
#Percentual por categoria
df_cat = (
    df_cat
    .join(df_total_cat, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .withColumn("pct_categoria", F.col("qtd_rec") / F.col("qtd_rec_total"))
)


In [104]:
#Categoria dominante
w = Window.partitionBy("NUM_CPF", "SAFRA_RECARGA").orderBy(F.desc("qtd_rec"))

df_dom = (
    df_cat
    .withColumn("rn", F.row_number().over(w))
    .filter(F.col("rn") == 1)
    .select(
        "NUM_CPF",
        "SAFRA_RECARGA",
        F.col("CAT_DSC_GRUPO_CARTAO").alias("DSC_GRUPO_CARTAO_DOMINANTE"),
        F.col("pct_categoria").alias("PCT_DSC_GRUPO_CARTAO_DOMINANTE")
    )
)

In [105]:
#Quantidade de categorias distintas
df_qtd = (
    df_cat
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.countDistinct("CAT_DSC_GRUPO_CARTAO")
        .alias("QTD_DSC_GRUPO_CARTAO_DISTINTOS")
    )
)


In [106]:
#Flags semânticas (binárias)
df_flags = (
    df_cat
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .pivot("CAT_DSC_GRUPO_CARTAO")
    .agg(F.sum("qtd_rec"))
    .fillna(0)
    .select(
        "NUM_CPF",
        "SAFRA_RECARGA",
        (F.col("RECARGA_ONLINE") > 0).cast("int").alias("FLG_RECARGA_ONLINE"),
        (F.col("ATIVACAO_PROMO") > 0).cast("int").alias("FLG_ATIVACAO_PROMO"),
        (F.col("CARTAO_FISICO") > 0).cast("int").alias("FLG_CARTAO_FISICO"),
        (F.col("PACOTE_SERVICO") > 0).cast("int").alias("FLG_PACOTE_SERVICO"),
        (F.col("CHIP") > 0).cast("int").alias("FLG_CHIP")
    )
)


In [107]:
#Dataset final de features
df_features_dsc_grupo_cartao = (
    df_dom
    .join(df_qtd, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_flags, ["NUM_CPF", "SAFRA_RECARGA"], "left")
)


| Variável                         | O que captura             |
| -------------------------------- | ------------------------- |
| `DSC_GRUPO_CARTAO_DOMINANTE`     | Tipo principal de recarga |
| `PCT_DSC_GRUPO_CARTAO_DOMINANTE` | Grau de concentração      |
| `QTD_DSC_GRUPO_CARTAO_DISTINTOS` | Diversidade               |
| `FLG_RECARGA_ONLINE`             | Digitalização             |
| `FLG_ATIVACAO_PROMO`             | Dependência de incentivo  |
| `FLG_CARTAO_FISICO`              | Canal físico              |
| `FLG_PACOTE_SERVICO`             | Consumo de pacotes        |
| `FLG_CHIP`                       | Aquisição / reposição     |


###COD_CANAL_AQUISICAO

In [108]:
from pyspark.sql import functions as F

df_dim_canal = (
    spark.read
    .option("header", True)
    .option("delimiter", ",")
    .csv("/content/drive/MyDrive/hackathon/base_recarga/BI_DIM_CANAL_AQUISICAO_CREDITO.csv")
)

In [109]:
df_dim_canal.show(5)

+-------------------+--------------------+--------------+------------------+------------------+----------------------+----------------------+------------------+------------------+----------------+--------------------+--------------------+
|COD_CANAL_AQUISICAO| DSC_CANAL_AQUISICAO|COD_SISTEMA_DW|DAT_ATUALIZACAO_DW|    DAT_CRIACAO_DW|COD_CANAL_AQUISICAO_BI|DSC_CANAL_AQUISICAO_BI|COD_AGENTE_CREDITO|  DAT_EXPIRACAO_DW|COD_TIPO_CREDITO|COD_TIPO_INSTITUICAO|DSC_TIPO_INSTITUICAO|
+-------------------+--------------------+--------------+------------------+------------------+----------------------+----------------------+------------------+------------------+----------------+--------------------+--------------------+
|              14929|SS/AA EBV - GOLFE...|            21|08JUL2020:08:22:50|16APR2015:22:19:21|                    -3|                    -3|              8210|08JUL2020:00:00:00|              CV|                NULL|                NULL|
|              14930|SS/AA EBV - MARAJ...|  

In [110]:
df_rec = (
    df_00
    .join(df_dim_canal, on="COD_CANAL_AQUISICAO", how="left")
)

In [111]:
df_rec = df_rec.withColumn(
    "canal_macro",
    F.when(F.col("COD_CANAL_AQUISICAO").isin(-2, -3), "NAO_INFORMADO")

     .when(F.lower(F.col("DSC_CANAL_AQUISICAO")).rlike("app|site|web|digital"),
           "DIGITAL")

     .when(F.lower(F.col("DSC_CANAL_AQUISICAO")).rlike("loja|pdv|varejo|banca"),
           "VAREJO")

     .when(F.lower(F.col("DSC_CANAL_AQUISICAO")).rlike("parceiro|convenio|terceiro"),
           "PARCEIRO")

     .when(F.lower(F.col("DSC_CANAL_AQUISICAO")).rlike("promoc|acao|campanha"),
           "PROMOCAO")

     .otherwise("OUTROS")
)

In [112]:
df_feat_canal = (
    df_rec
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        F.count("*").alias("qtd_rec_total"),

        F.sum(F.when(F.col("canal_macro") == "DIGITAL", 1).otherwise(0))
            .alias("qtd_rec_digital"),

        F.sum(F.when(F.col("canal_macro") == "VAREJO", 1).otherwise(0))
            .alias("qtd_rec_varejo"),

        F.sum(F.when(F.col("canal_macro") == "PROMOCAO", 1).otherwise(0))
            .alias("qtd_rec_promocao"),

        F.countDistinct("canal_macro").alias("qtd_canais_distintos")
    )
)


In [113]:
df_feat_canal = (
    df_feat_canal
    .withColumn(
        "pct_rec_digital",
        F.col("qtd_rec_digital") / F.col("qtd_rec_total")
    )
    .withColumn(
        "pct_rec_varejo",
        F.col("qtd_rec_varejo") / F.col("qtd_rec_total")
    )
    .withColumn(
        "flg_canal_diversificado",
        F.when(F.col("qtd_canais_distintos") > 1, 1).otherwise(0)
    )
)
df_feat_canal = df_feat_canal.drop("qtd_rec_total","qtd_rec_digital","qtd_rec_varejo")

| Feature                   | Por que é boa                  |
| ------------------------- | ------------------------------ |
| `pct_rec_digital`         | Proxy de maturidade digital    |
| `pct_rec_varejo`          | Dependência de canal físico    |
| `qtd_canais_distintos`    | Flexibilidade de comportamento |
| `flg_canal_diversificado` | Resiliência / engajamento      |
| `qtd_rec_promocao`        | Sensibilidade a incentivo      |


O campo COD_CANAL_AQUISICAO apresenta alta cardinalidade e elevada taxa de não preenchimento. Para fins analíticos, os canais foram consolidados em macro-categorias com base em sua natureza operacional, permitindo a extração de indicadores robustos de preferência de canal, diversificação e maturidade digital do cliente.

###COD_PLATAFORMA_ATU

In [114]:
# df_recarga_por_plataforma = (
#     df_00.groupBy("COD_PLATAFORMA_ATU")
#     .agg(count("*").alias("qtd"))

# )
# df_recarga_por_plataforma.sort("qtd").show()

Devido à elevada concentração da base em poucos códigos de plataforma, a variável original não foi utilizada diretamente. Foram derivadas flags macro de segmento, engajamento digital, recorrência (autocontrole) e risco, além de indicadores de estabilidade e migração entre plataformas, garantindo robustez estatística e interpretabilidade para o modelo.

In [115]:
from pyspark.sql import functions as F
from pyspark.sql import Window


In [116]:
df_base = (
    df_00
    # -------- Segmentos macro --------
    .withColumn("flg_pre_pago",
                F.when(F.col("COD_PLATAFORMA_ATU").startswith("PRE"), 1).otherwise(0))

    .withColumn("flg_pos_pago",
                F.when(F.col("COD_PLATAFORMA_ATU").startswith("POS"), 1).otherwise(0))

    .withColumn("flg_controle",
                F.when(F.col("COD_PLATAFORMA_ATU").startswith("CTL"), 1).otherwise(0))

    .withColumn("flg_flex",
                F.when(F.col("COD_PLATAFORMA_ATU").isin("FLEXD", "AUTOC"), 1).otherwise(0))

    # -------- Engajamento / digital --------
    .withColumn("flg_autocontrole",
                F.when(F.col("COD_PLATAFORMA_ATU") == "AUTOC", 1).otherwise(0))

    .withColumn("flg_digital",
                F.when(F.col("COD_PLATAFORMA_ATU")
                       .isin("FLEXD", "AUTOC", "MVNOD"), 1).otherwise(0))

    # -------- Risco --------
    .withColumn("flg_bloqueado",
                F.when(F.col("COD_PLATAFORMA_ATU")
                       .isin("PREBL", "POSBL", "CTLBL"), 1).otherwise(0))

    .withColumn("flg_restricao_credito",
                F.when(F.col("COD_PLATAFORMA_ATU")
                       .isin("POSBC", "POSCW", "POSRI"), 1).otherwise(0))

    # -------- Exclusão --------
    .withColumn("flg_nao_humano",
                F.when(F.col("COD_PLATAFORMA_ATU")
                       .isin("M2MS", "SGIOT"), 1).otherwise(0))

    .withColumn("flg_tecnico",
                F.when(F.col("COD_PLATAFORMA_ATU")
                       .isin("-1", "-2", "-3"), 1).otherwise(0))
)


In [117]:
df_feat_cod_plataforma_atu = (
    df_base
    .groupBy("NUM_CPF", "SAFRA_RECARGA")
    .agg(
        # Contagem total
        F.count("*").alias("qtd_eventos"),

        # Intensidade
        F.sum("flg_pre_pago").alias("qtd_pre_pago"),
        F.sum("flg_flex").alias("qtd_flex"),
        F.sum("flg_autocontrole").alias("qtd_autocontrole"),
        F.sum("flg_digital").alias("qtd_digital"),

        # Risco
        F.max("flg_bloqueado").alias("flg_bloqueado"),
        F.max("flg_restricao_credito").alias("flg_restricao_credito"),

        # Exclusão
        F.max("flg_nao_humano").alias("flg_nao_humano"),
        F.max("flg_tecnico").alias("flg_tecnico"),

        # Estabilidade
        F.countDistinct("COD_PLATAFORMA_ATU").alias("qtd_plataformas_distintas")
    )
)


In [118]:
df_feat_cod_plataforma_atu = (
    df_feat_cod_plataforma_atu
    .withColumn("pct_pre_pago",
                F.col("qtd_pre_pago") / F.col("qtd_eventos"))

    .withColumn("pct_flex",
                F.col("qtd_flex") / F.col("qtd_eventos"))

    .withColumn("pct_autocontrole",
                F.col("qtd_autocontrole") / F.col("qtd_eventos"))

    .withColumn("pct_digital",
                F.col("qtd_digital") / F.col("qtd_eventos"))
)


In [119]:
w = Window.partitionBy("NUM_CPF").orderBy("SAFRA_RECARGA")

df_feat_cod_plataforma_atu = (
    df_feat_cod_plataforma_atu
    .withColumn("pct_flex_lag",
                F.lag("pct_flex").over(w))

    .withColumn("flg_migracao_plataforma",
                F.when(
                    (F.col("pct_flex_lag").isNotNull()) &
                    (F.col("pct_flex") != F.col("pct_flex_lag")),
                    1
                ).otherwise(0))

    .withColumn("flg_upgrade_flex",
                F.when(
                    (F.col("pct_flex_lag") < 0.5) &
                    (F.col("pct_flex") >= 0.5),
                    1
                ).otherwise(0))

    .withColumn("flg_downgrade_flex",
                F.when(
                    (F.col("pct_flex_lag") >= 0.5) &
                    (F.col("pct_flex") < 0.5),
                    1
                ).otherwise(0))
)


In [120]:
df_feat_cod_plataforma_atu = (
    df_feat_cod_plataforma_atu
    .withColumn(
        "flg_excluir_modelo",
        F.when(
            (F.col("flg_nao_humano") == 1) |
            (F.col("flg_tecnico") == 1),
            1
        ).otherwise(0)
    )
)


#Book consolidado

In [121]:
# Consolidation of the book, avoiding duplicate columns
df_book = (
    df_num # Contains NUM_CPF, SAFRA_RECARGA, dt_ultima_recarga, vlr_total_bonus, vlr_medio_bonus initially
    .join(df_fx_hora, ["NUM_CPF", "SAFRA_RECARGA"], "left") # Adds qtd_rec_madrugada, etc.
    .join(df_hora_dom, ["NUM_CPF", "SAFRA_RECARGA"], "left") # Adds fx_hora_dominante
    .join(df_sos, ["NUM_CPF", "SAFRA_RECARGA"], "left") # Adds qtd_rec_sos, vlr_medio_rec_sos, pct_rec_sos, flg_uso_sos
    .join(
        df_bonus.drop("vlr_total_bonus", "vlr_medio_bonus"), # Drop these from df_bonus as they are already in df_num
        ["NUM_CPF", "SAFRA_RECARGA"],
        "left"
    ) # Adds qtd_rec_com_bonus, vlr_rec_com_bonus, pct_rec_com_bonus, fx_dependencia_bonus
    .join(df_canal, ["NUM_CPF", "SAFRA_RECARGA"], "left") # Adds canal_principal
    .join(
        df_recencia_enriq.drop("dt_ultima_recarga", "dt_ref"), # Drop these from df_recencia_enriq as dt_ultima_recarga is already in df_num and dt_ref is not a feature
        ["NUM_CPF", "SAFRA_RECARGA"],
        "left"
    ) # Adds recencia_dias, fx_recencia, gap_medio_recarga, gap_std_recarga, desvio_recencia, flg_quebra_padrao, score_risco_recencia
    .join(df_plano, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_tipo_insercao, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_forma_pagamento_counts, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_feat, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_feat_canal, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_feat_cod_plataforma_atu, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_status_feat, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_instituicao_fim, ["NUM_CPF", "SAFRA_RECARGA"], "left").drop("QTD_RECARGAS_TOTAL")
    .join(df_features_grupo_cartao, ["NUM_CPF", "SAFRA_RECARGA"], "left")
    .join(df_features_dsc_grupo_cartao, ["NUM_CPF", "SAFRA_RECARGA"], "left")

)


# Now, after all base features are in df_book, calculate the combined indicators
df_book = df_book.withColumn(
    "perfil_emergencial",
    F.when(
        (F.col("recencia_dias") <= 7) & (F.col("flg_uso_sos") == 1),
        1
    ).otherwise(0)
)

df_book = df_book.withColumn(
    "perfil_incentivado",
    F.when(
        (F.col("recencia_dias") > 30) & (F.col("fx_dependencia_bonus").isin("03_Media", "04_Alta")),
        1
    ).otherwise(0)
)

In [122]:
df_book.printSchema()

root
 |-- NUM_CPF: string (nullable = true)
 |-- SAFRA_RECARGA: string (nullable = true)
 |-- qtd_recargas: long (nullable = false)
 |-- vlr_total_recarga: double (nullable = true)
 |-- vlr_medio_recarga: double (nullable = true)
 |-- vlr_max_recarga: double (nullable = true)
 |-- vlr_min_recarga: double (nullable = true)
 |-- qtd_telefones: long (nullable = false)
 |-- dt_primeira_recarga: date (nullable = true)
 |-- dt_ultima_recarga: date (nullable = true)
 |-- vlr_total_bonus: double (nullable = true)
 |-- vlr_medio_bonus: double (nullable = true)
 |-- vlr_total_real: double (nullable = true)
 |-- vlr_medio_real: double (nullable = true)
 |-- qtd_rec_madrugada: long (nullable = true)
 |-- qtd_rec_manha: long (nullable = true)
 |-- qtd_rec_tarde: long (nullable = true)
 |-- qtd_rec_noite: long (nullable = true)
 |-- fx_hora_dominante: string (nullable = true)
 |-- qtd_rec_sos: double (nullable = true)
 |-- vlr_medio_rec_sos: double (nullable = true)
 |-- pct_rec_sos: double (nullabl

In [123]:
#quantidade de variáveis do book
len(df_book.columns)

114

In [124]:
#df_book.select("NUM_CPF").distinct().count()

In [125]:
#df_book.filter(F.col('NUM_CPF') == 'ZZX7WZZ7YZX').show()

# Construção das janelas temporárias de 1, 3. 6, 9, 12 meses

## Função genérica para janelas temporais VERSÃO COMPLETA

In [322]:
def window_features(df, months):

    w = Window.partitionBy("NUM_CPF") \
              .orderBy("SAFRA_RECARGA") \
              .rowsBetween(-months + 1, 0)

    df_w = df.select (
        "NUM_CPF",
        "SAFRA_RECARGA",

        # =========================
        # INTENSIDADE/MÉDIA (nível médio)
        # =========================
        #   F.avg("qtd_recargas").over(w).alias(f"mean_qtd_recargas_u{months}m"),
        # #F.avg("vlr_total_recarga").over(w).alias(f"mean_vlr_total_recarga_u{months}m"),
        #   F.avg("vlr_medio_recarga").over(w).alias(f"mean_medio_recarga_u{months}m"),
        # F.avg("vlr_max_recarga").over(w).alias(f"mean_vlr_max_recarga_u{months}m"),
        # F.avg("vlr_min_recarga").over(w).alias(f"mean_vlr_min_recarga_u{months}m"),
        # F.avg("qtd_telefones").over(w).alias(f"mean_qtd_telefones_u{months}m"),
        # F.avg("vlr_total_bonus").over(w).alias(f"mean_vlr_total_bonus_u{months}m"),
        # F.avg("vlr_medio_bonus").over(w).alias(f"mean_vlr_medio_bonus_u{months}m"),
        # F.avg("qtd_rec_madrugada").over(w).alias(f"mean_qtd_rec_madrugada_u{months}m"),
        # F.avg("qtd_rec_manha").over(w).alias(f"mean_qtd_rec_manha_u{months}m"),
        # F.avg("qtd_rec_tarde").over(w).alias(f"mean_qtd_rec_tarde_u{months}m"),
        # F.avg("qtd_rec_noite").over(w).alias(f"mean_qtd_rec_noite_u{months}m"),
        # F.avg("qtd_rec_sos").over(w).alias(f"mean_qtd_rec_sos_u{months}m"),
          F.avg("pct_rec_sos").over(w).alias(f"mean_pct_rec_sos_u{months}m"),
        # F.avg("qtd_rec_com_bonus").over(w).alias(f"mean_qtd_rec_com_bonus_u{months}m"),
          F.avg("pct_rec_com_bonus").over(w).alias(f"mean_pct_rec_com_bonus_u{months}m"),
          F.avg("gap_medio_recarga").over(w).alias(f"mean_gap_medio_recarga_u{months}m"),
        #   F.avg("gap_std_recarga").over(w).alias(f"mean_gap_std_recarga_u{months}m"),
        # F.avg("desvio_recencia").over(w).alias(f"mean_desvio_recencia_u{months}m"),
        # F.avg("score_risco_recencia").over(w).alias(f"mean_score_risco_recencia_u{months}m"),
        # F.avg("qtd_rec_plano_541200").over(w).alias(f"mean_qtd_rec_plano_541200_u{months}m"),
        # F.avg("qtd_rec_plano_668900").over(w).alias(f"mean_qtd_rec_plano_668900_u{months}m"),
        # F.avg("qtd_rec_plano_599100").over(w).alias(f"mean_qtd_rec_plano_599100_u{months}m"),
        # F.avg("qtd_rec_plano_607690").over(w).alias(f"mean_qtd_rec_plano_607690_u{months}m"),
        # F.avg("qtd_rec_plano_999999").over(w).alias(f"mean_qtd_rec_plano_999999_u{months}m"),
        # F.avg("qtd_rec_plano_369889").over(w).alias(f"mean_qtd_rec_plano_369889_u{months}m"),
        # F.avg("qtd_rec_plano_393401").over(w).alias(f"mean_qtd_rec_plano_393401_u{months}m"),
        # F.avg("qtd_tipo_insercao").over(w).alias(f"mean_qtd_tipo_insercao_u{months}m"),
        # F.avg("qtd_forma_pgto_na").over(w).alias(f"mean_qtd_forma_pgto_na_u{months}m"),
        # F.avg("qtd_forma_pgto_nd").over(w).alias(f"mean_qtd_forma_pgto_nd_u{months}m"),
        # F.avg("qtd_forma_pgto_ni").over(w).alias(f"mean_qtd_forma_pgto_ni_u{months}m"),
        # F.avg("qtd_forma_pgto_cp").over(w).alias(f"mean_qtd_forma_pgto_cp_u{months}m"),
        # F.avg("qtd_forma_pgto_dd").over(w).alias(f"mean_qtd_forma_pgto_dd_u{months}m"),
        # F.avg("qtd_forma_pgto_mp").over(w).alias(f"mean_qtd_forma_pgto_mp_u{months}m"),
        # F.avg("qtd_forma_pgto_pb").over(w).alias(f"mean_qtd_forma_pgto_pb_u{months}m"),
        # F.avg("qtd_forma_pgto_ca").over(w).alias(f"mean_qtd_forma_pgto_ca_u{months}m"),
        # F.avg("qtd_forma_pgto_pa").over(w).alias(f"mean_qtd_forma_pgto_pa_u{months}m"),
        # F.avg("qtd_rec_promocao").over(w).alias(f"mean_qtd_rec_promocao_u{months}m"),
        #   F.avg("qtd_canais_distintos").over(w).alias(f"mean_qtd_canais_distintos_u{months}m"),
        #   F.avg("pct_rec_digital").over(w).alias(f"mean_pct_rec_digital_u{months}m"),
        # F.avg("pct_rec_varejo").over(w).alias(f"mean_pct_rec_varejo_u{months}m"),
        # F.avg("qtd_eventos").over(w).alias(f"mean_qtd_eventos_u{months}m"),
        # F.avg("qtd_pre_pago").over(w).alias(f"mean_qtd_pre_pago_u{months}m"),
        # F.avg("qtd_flex").over(w).alias(f"mean_qtd_flex_u{months}m"),
        # F.avg("qtd_autocontrole").over(w).alias(f"mean_qtd_autocontrole_u{months}m"),
        # F.avg("qtd_digital").over(w).alias(f"mean_qtd_digital_u{months}m"),
        # F.avg("qtd_plataformas_distintas").over(w).alias(f"mean_qtd_plataformas_distintas_u{months}m"),
        # F.avg("pct_pre_pago").over(w).alias(f"mean_pct_pre_pago_u{months}m"),
        # F.avg("pct_flex").over(w).alias(f"mean_pct_flex_u{months}m"),
        # F.avg("pct_autocontrole").over(w).alias(f"mean_pct_autocontrole_u{months}m"),
        # F.avg("pct_digital").over(w).alias(f"mean_pct_digital_u{months}m"),
        # F.avg("pct_flex_lag").over(w).alias(f"mean_pct_flex_lag_u{months}m"),
        # F.avg("QTD_RECARGAS_DIGITAL").over(w).alias(f"mean_QTD_RECARGAS_DIGITAL_u{months}m"),
        # F.avg("QTD_RECARGAS_BANCO").over(w).alias(f"mean_QTD_RECARGAS_BANCO_u{months}m"),
        # F.avg("QTD_RECARGAS_VAREJO").over(w).alias(f"mean_QTD_RECARGAS_VAREJO_u{months}m"),
        # F.avg("QTD_RECARGAS_TELECOM").over(w).alias(f"mean_QTD_RECARGAS_TELECOM_u{months}m"),
        # F.avg("PCT_RECARGA_DIGITAL").over(w).alias(f"mean_PCT_RECARGA_DIGITAL_u{months}m"),
        # F.avg("PCT_RECARGA_BANCO").over(w).alias(f"mean_PCT_RECARGA_BANCO_u{months}m"),
        # F.avg("PCT_RECARGA_VAREJO").over(w).alias(f"mean_PCT_RECARGA_VAREJO_u{months}m"),
        # F.avg("PCT_RECARGA_TELECOM").over(w).alias(f"mean_PCT_RECARGA_TELECOM_u{months}m"),
        # F.avg("PCT_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"mean_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # F.avg("QTD_GRUPOS_CARTAO_DISTINTOS").over(w).alias(f"mean_QTD_GRUPOS_CARTAO_DISTINTOS_u{months}m"),
        # F.avg("PCT_DSC_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"mean_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # F.avg("QTD_DSC_GRUPO_CARTAO_DISTINTOS").over(w).alias(f"mean_QTD_DSC_GRUPO_CARTAO_DISTINTOS_u{months}m"),
          F.avg("recencia_dias").over(w).alias(f"mean_recencia_dias_u{months}m"),


        # =========================
        #  FREQUENCIA/CONTAGEM
        # =========================
        # # qtd_recargas_u_m contabiliza a quantidade de recargas no periodo dado
         F.count("qtd_recargas").over(w).alias(f"qtd_recargas_u{months}m"),
        # # qtd_telefones_u_m contabiliza a quantidade de telefones no periodo dado
        #  F.count("qtd_telefones").over(w).alias(f"qtd_telefones_u{months}m"),
        # # qtd_rec_madrugada_u_m
        # F.count("qtd_rec_madrugada").over(w).alias(f"qtd_rec_madrugada_u{months}m"),
        # # qtd_rec_manha_u_m
        # F.count("qtd_rec_manha").over(w).alias(f"qtd_rec_manha_u{months}m"),
        # # qtd_rec_tarde_u_m
        # F.count("qtd_rec_tarde").over(w).alias(f"qtd_rec_tarde_u{months}m"),
        # # qtd_rec_noite_u_m
        # F.count("qtd_rec_noite").over(w).alias(f"qtd_rec_noite_u{months}m"),
        # # qtd_rec_sos_u_m contabiliza a quantidade de recargas sos no periodo dado
         F.count("qtd_rec_sos").over(w).alias(f"qtd_rec_sos_u{months}m"),
        # # qtd_rec_com_bonus contabiliza a quantidade de recargas com bonus no periodo dado
        #  F.count("qtd_rec_com_bonus").over(w).alias(f"qtd_rec_com_bonus_u{months}m"),
        # # qtd_rec_plano_541200
        # F.count("qtd_rec_plano_541200").over(w).alias(f"qtd_rec_plano_541200_u{months}m"),
        # # qtd_rec_plano_668900
        # F.count("qtd_rec_plano_668900").over(w).alias(f"qtd_rec_plano_668900_u{months}m"),
        # # qtd_rec_plano_599100
        # F.count("qtd_rec_plano_599100").over(w).alias(f"qtd_rec_plano_599100_u{months}m"),
        # # qtd_rec_plano_607690
        # F.count("qtd_rec_plano_607690").over(w).alias(f"qtd_rec_plano_607690_u{months}m"),
        # # qtd_rec_plano_999999
        # F.count("qtd_rec_plano_999999").over(w).alias(f"qtd_rec_plano_999999_u{months}m"),
        # # qtd_rec_plano_369889
        # F.count("qtd_rec_plano_369889").over(w).alias(f"qtd_rec_plano_369889_u{months}m"),
        # # qtd_rec_plano_393401
        # F.count("qtd_rec_plano_393401").over(w).alias(f"qtd_rec_plano_393401_u{months}m"),
        # # qtd_tipo_insercao
        # F.count("qtd_tipo_insercao").over(w).alias(f"qtd_tipo_insercao_u{months}m"),
        # # qtd_forma_pgto_na
        # F.count("qtd_forma_pgto_na").over(w).alias(f"qtd_forma_pgto_na_u{months}m"),
        # # qtd_forma_pgto_nd
        # F.count("qtd_forma_pgto_nd").over(w).alias(f"qtd_forma_pgto_nd_u{months}m"),
        # # qtd_forma_pgto_ni
        # F.count("qtd_forma_pgto_ni").over(w).alias(f"qtd_forma_pgto_ni_u{months}m"),
        # # qtd_forma_pgto_cp
        # F.count("qtd_forma_pgto_cp").over(w).alias(f"qtd_forma_pgto_cp_u{months}m"),
        # # qtd_forma_pgto_dd
        # F.count("qtd_forma_pgto_dd").over(w).alias(f"qtd_forma_pgto_dd_u{months}m"),
        # # qtd_forma_pgto_mp
        # F.count("qtd_forma_pgto_mp").over(w).alias(f"qtd_forma_pgto_mp_u{months}m"),
        # # qtd_forma_pgto_pb
        # F.count("qtd_forma_pgto_pb").over(w).alias(f"qtd_forma_pgto_pb_u{months}m"),
        # # qtd_forma_pgto_ca
        # F.count("qtd_forma_pgto_ca").over(w).alias(f"qtd_forma_pgto_ca_u{months}m"),
        # # qtd_forma_pgto_pa
        # F.count("qtd_forma_pgto_pa").over(w).alias(f"qtd_forma_pgto_pa_u{months}m"),
        # # qtd_recarga_adicional_controle
        # F.count("qtd_recarga_adicional_controle").over(w).alias(f"qtd_recarga_adicional_controle_u{months}m"),
        # # qtd_rec_promocao
        # F.count("qtd_rec_promocao").over(w).alias(f"qtd_rec_promocao_u{months}m"),
        # # qtd_canais_distintos
         F.count("qtd_canais_distintos").over(w).alias(f"qtd_canais_distintos_u{months}m"),
        # # qtd_eventos
         F.count("qtd_eventos").over(w).alias(f"qtd_eventos_u{months}m"),
        # # qtd_pre_pago
        # F.count("qtd_pre_pago").over(w).alias(f"qtd_pre_pago_u{months}m"),
        # # qtd_flex
        # F.count("qtd_flex").over(w).alias(f"qtd_flex_u{months}m"),
        # # qtd_autocontrole
        # F.count("qtd_autocontrole").over(w).alias(f"qtd_autocontrole_u{months}m"),
        # # qtd_digital
        #  F.count("qtd_digital").over(w).alias(f"qtd_digital_u{months}m"),
        # # qtd_plataformas_distintas
        #  F.count("qtd_plataformas_distintas").over(w).alias(f"qtd_plataformas_distintas_u{months}m"),
        # # qtd_status_distintos
        #  F.count("qtd_status_distintos").over(w).alias(f"qtd_status_distintos_u{months}m"),
        # # QTD_RECARGAS_DIGITAL
        # F.count("QTD_RECARGAS_DIGITAL").over(w).alias(f"QTD_RECARGAS_DIGITAL_u{months}m"),
        # # QTD_RECARGAS_BANCO
        # F.count("QTD_RECARGAS_BANCO").over(w).alias(f"QTD_RECARGAS_BANCO_u{months}m"),
        # # QTD_RECARGAS_VAREJO
        # F.count("QTD_RECARGAS_VAREJO").over(w).alias(f"QTD_RECARGAS_VAREJO_u{months}m"),
        # # QTD_RECARGAS_TELECOM
        # F.count("QTD_RECARGAS_TELECOM").over(w).alias(f"QTD_RECARGAS_TELECOM_u{months}m"),
        # # QTD_GRUPOS_CARTAO_DISTINTOS
        # F.count("QTD_GRUPOS_CARTAO_DISTINTOS").over(w).alias(f"QTD_GRUPOS_CARTAO_DISTINTOS_u{months}m"),
        # # QTD_DSC_GRUPO_CARTAO_DISTINTOS
        # F.count("QTD_DSC_GRUPO_CARTAO_DISTINTOS").over(w).alias(f"QTD_DSC_GRUPO_CARTAO_DISTINTOS_u{months}m"),
        # # recencia_dias
        # F.count("recencia_dias").over(w).alias(f"recencia_dias_u{months}m"),


        # =========================
        # VOLATILIDADE/VARIABILIDADE
        # =========================
        # F.stddev("vlr_total_recarga").over(w).alias(f"std_vlr_total_recarga_u{months}m"),
          F.stddev("vlr_medio_recarga").over(w).alias(f"std_vlr_medio_recarga_u{months}m"),
        # F.stddev("vlr_max_recarga").over(w).alias(f"std_vlr_max_recarga_u{months}m"),
        # F.stddev("vlr_min_recarga").over(w).alias(f"std_vlr_min_recarga_u{months}m"),
        # F.stddev("vlr_total_bonus").over(w).alias(f"std_vlr_total_bonus_u{months}m"),
        # F.stddev("vlr_medio_bonus").over(w).alias(f"std_vlr_medio_bonus_u{months}m"),
        # F.stddev("vlr_total_real").over(w).alias(f"std_vlr_total_real_u{months}m"),
        # F.stddev("vlr_medio_real").over(w).alias(f"std_vlr_medio_real_u{months}m"),
        # F.stddev("vlr_medio_rec_sos").over(w).alias(f"std_vlr_medio_rec_sos_u{months}m"),
        #  F.stddev("pct_rec_sos").over(w).alias(f"std_pct_rec_sos_u{months}m"),
        # F.stddev("vlr_rec_com_bonus").over(w).alias(f"std_vlr_rec_com_bonus_u{months}m"),
          F.stddev("pct_rec_com_bonus").over(w).alias(f"std_pct_rec_com_bonus_u{months}m"),
         F.stddev("gap_medio_recarga").over(w).alias(f"std_gap_medio_recarga_u{months}m"),
        # F.stddev("gap_std_recarga").over(w).alias(f"std_gap_std_recarga_u{months}m"),
        #  F.stddev("desvio_recencia").over(w).alias(f"std_desvio_recencia_u{months}m"),
        #  F.stddev("score_risco_recencia").over(w).alias(f"std_score_risco_recencia_u{months}m"),
        # F.stddev("pct_recarga_adicional_controle").over(w).alias(f"std_pct_recarga_adicional_controle_u{months}m"),
        #  F.stddev("pct_rec_digital").over(w).alias(f"std_pct_rec_digital_u{months}m"),
        # F.stddev("pct_rec_varejo").over(w).alias(f"std_pct_rec_varejo_u{months}m"),
        # F.stddev("pct_pre_pago").over(w).alias(f"std_pct_pre_pago_u{months}m"),
        # F.stddev("pct_flex").over(w).alias(f"std_pct_flex_u{months}m"),
        # F.stddev("pct_autocontrole").over(w).alias(f"std_pct_autocontrole_u{months}m"),
        # F.stddev("pct_digital").over(w).alias(f"std_pct_digital_u{months}m"),
        # F.stddev("pct_flex_lag").over(w).alias(f"std_pct_flex_lag_u{months}m"),
        #  F.stddev("pct_linha_ativa").over(w).alias(f"std_pct_linha_ativa_u{months}m"),
        #  F.stddev("pct_expirado_recente").over(w).alias(f"std_pct_expirado_recente_u{months}m"),
        # F.stddev("PCT_RECARGA_DIGITAL").over(w).alias(f"std_PCT_RECARGA_DIGITAL_u{months}m"),
        # F.stddev("PCT_RECARGA_BANCO").over(w).alias(f"std_PCT_RECARGA_BANCO_u{months}m"),
        # F.stddev("PCT_RECARGA_VAREJO").over(w).alias(f"std_PCT_RECARGA_VAREJO_u{months}m"),
        # F.stddev("PCT_RECARGA_TELECOM").over(w).alias(f"std_PCT_RECARGA_TELECOM_u{months}m"),
        # F.stddev("PCT_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"std_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # F.stddev("PCT_DSC_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"std_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
         F.stddev("recencia_dias").over(w).alias(f"std_recencia_dias_u{months}m"),


        # =========================
        # VOLUME/SOMA
        # =========================
        # Quantidade acumulada de recargas
         F.sum("qtd_recargas").over(w).alias(f"sum_qtd_recargas_u{months}m"),
        # # Soma acumulada de recargas total
         F.sum("vlr_total_recarga").over(w).alias(f"sum_vlr_recarga_u{months}m"),
        # # Soma acumulada de recargas médias
        # F.sum("vlr_medio_recarga").over(w).alias(f"sum_vlr_medio_recarga_u{months}m"),
        # # vlr_max_recarga
        # F.sum("vlr_max_recarga").over(w).alias(f"sum_vlr_max_recarga_u{months}m"),
        # # vlr_min_recarga
        # F.sum("vlr_min_recarga").over(w).alias(f"sum_vlr_min_recarga_u{months}m"),
        # # qtd_telefones
        # F.sum("qtd_telefones").over(w).alias(f"sum_qtd_telefones_u{months}m"),
        # # vlr_total_bonus
         F.sum("vlr_total_bonus").over(w).alias(f"sum_vlr_total_bonus_u{months}m"),
        # # vlr_medio_bonus
        # F.sum("vlr_medio_bonus").over(w).alias(f"sum_vlr_medio_bonus_u{months}m"),
        # # vlr_total_real
         F.sum("vlr_total_real").over(w).alias(f"sum_vlr_total_real_u{months}m"),
        # # vlr_medio_real
        # F.sum("vlr_medio_real").over(w).alias(f"sum_vlr_medio_real_u{months}m"),
        # # qtd_rec_madrugada
        # F.sum("qtd_rec_madrugada").over(w).alias(f"sum_qtd_rec_madrugada_u{months}m"),
        # # qtd_rec_manha
        # F.sum("qtd_rec_manha").over(w).alias(f"sum_qtd_rec_manha_u{months}m"),
        # # qtd_rec_tarde
        # F.sum("qtd_rec_tarde").over(w).alias(f"sum_qtd_rec_tarde_u{months}m"),
        # # qtd_rec_noite
        # F.sum("qtd_rec_noite").over(w).alias(f"sum_qtd_rec_noite_u{months}m"),
        # # qtd_rec_sos
         F.sum("qtd_rec_sos").over(w).alias(f"sum_qtd_rec_sos_u{months}m"),
        # # vlr_medio_rec_sos
        #  F.sum("vlr_medio_rec_sos").over(w).alias(f"sum_vlr_medio_rec_sos_u{months}m"),
        # # pct_rec_sos
        # F.sum("pct_rec_sos").over(w).alias(f"sum_pct_rec_sos_u{months}m"),
        # # qtd_rec_com_bonus
         F.sum("qtd_rec_com_bonus").over(w).alias(f"sum_qtd_rec_com_bonus_u{months}m"),
        # # Soma acumulada de recargas com bonus
        # F.sum("vlr_rec_com_bonus").over(w).alias(f"sum_vlr_rec_com_bonus_u{months}m"),
        # # pct_rec_com_bonus
        # F.sum("pct_rec_com_bonus").over(w).alias(f"sum_pct_rec_com_bonus_u{months}m"),
        # # gap_medio_recarga
        # F.sum("gap_medio_recarga").over(w).alias(f"sum_gap_medio_recarga_u{months}m"),
        # # gap_std_recarga
        # F.sum("gap_std_recarga").over(w).alias(f"sum_gap_std_recarga_u{months}m"),
        # # desvio_recencia
        # F.sum("desvio_recencia").over(w).alias(f"sum_desvio_recencia_u{months}m"),
        # # score_risco_recencia
        # F.sum("score_risco_recencia").over(w).alias(f"sum_score_risco_recencia_u{months}m"),
        # # qtd_rec_plano_541200
        # F.sum("qtd_rec_plano_541200").over(w).alias(f"sum_qtd_rec_plano_541200_u{months}m"),
        # # qtd_rec_plano_668900
        # F.sum("qtd_rec_plano_668900").over(w).alias(f"sum_qtd_rec_plano_668900_u{months}m"),
        # # qtd_rec_plano_599100
        # F.sum("qtd_rec_plano_599100").over(w).alias(f"sum_qtd_rec_plano_599100_u{months}m"),
        # # qtd_rec_plano_607690
        # F.sum("qtd_rec_plano_607690").over(w).alias(f"sum_qtd_rec_plano_607690_u{months}m"),
        # # qtd_rec_plano_999999
        # F.sum("qtd_rec_plano_999999").over(w).alias(f"sum_qtd_rec_plano_999999_u{months}m"),
        # # qtd_rec_plano_369889
        # F.sum("qtd_rec_plano_369889").over(w).alias(f"sum_qtd_rec_plano_369889_u{months}m"),
        # # qtd_rec_plano_393401
        # F.sum("qtd_rec_plano_393401").over(w).alias(f"sum_qtd_rec_plano_393401_u{months}m"),
        # # qtd_tipo_insercao
        # F.sum("qtd_tipo_insercao").over(w).alias(f"sum_qtd_tipo_insercao_u{months}m"),
        # # qtd_forma_pgto_na
        # F.sum("qtd_forma_pgto_na").over(w).alias(f"sum_qtd_forma_pgto_na_u{months}m"),
        # # qtd_forma_pgto_nd
        # F.sum("qtd_forma_pgto_nd").over(w).alias(f"sum_qtd_forma_pgto_nd_u{months}m"),
        # # qtd_forma_pgto_ni
        # F.sum("qtd_forma_pgto_ni").over(w).alias(f"sum_qtd_forma_pgto_ni_u{months}m"),
        # # qtd_forma_pgto_cp
        # F.sum("qtd_forma_pgto_cp").over(w).alias(f"sum_qtd_forma_pgto_cp_u{months}m"),
        # # qtd_forma_pgto_dd
        # F.sum("qtd_forma_pgto_dd").over(w).alias(f"sum_qtd_forma_pgto_dd_u{months}m"),
        # # qtd_forma_pgto_mp
        # F.sum("qtd_forma_pgto_mp").over(w).alias(f"sum_qtd_forma_pgto_mp_u{months}m"),
        # # qtd_forma_pgto_pb
        # F.sum("qtd_forma_pgto_pb").over(w).alias(f"sum_qtd_forma_pgto_pb_u{months}m"),
        # # qtd_forma_pgto_ca
        # F.sum("qtd_forma_pgto_ca").over(w).alias(f"sum_qtd_forma_pgto_ca_u{months}m"),
        # # qtd_forma_pgto_pa
        # F.sum("qtd_forma_pgto_pa").over(w).alias(f"sum_qtd_forma_pgto_pa_u{months}m"),
        # # qtd_recarga_adicional_controle
        # F.sum("qtd_recarga_adicional_controle").over(w).alias(f"sum_qtd_recarga_adicional_controle_u{months}m"),
        # # pct_recarga_adicional_controle
        # F.sum("pct_recarga_adicional_controle").over(w).alias(f"sum_pct_recarga_adicional_controle_u{months}m"),
        # # qtd_rec_promocao
        # F.sum("qtd_rec_promocao").over(w).alias(f"sum_qtd_rec_promocao_u{months}m"),
        # # qtd_canais_distintos
        # F.sum("qtd_canais_distintos").over(w).alias(f"sum_qtd_canais_distintos_u{months}m"),
        # # pct_rec_digital
        # F.sum("pct_rec_digital").over(w).alias(f"sum_pct_rec_digital_u{months}m"),
        # # pct_rec_varejo
        # F.sum("pct_rec_varejo").over(w).alias(f"sum_pct_rec_varejo_u{months}m"),
        # # qtd_eventos
         F.sum("qtd_eventos").over(w).alias(f"sum_qtd_eventos_u{months}m"),
        # # qtd_pre_pago
        # F.sum("qtd_pre_pago").over(w).alias(f"sum_qtd_pre_pago_u{months}m"),
        # # qtd_flex
        # F.sum("qtd_flex").over(w).alias(f"sum_qtd_flex_u{months}m"),
        # # qtd_autocontrole
        # F.sum("qtd_autocontrole").over(w).alias(f"sum_qtd_autocontrole_u{months}m"),
        # # qtd_digital
        # F.sum("qtd_digital").over(w).alias(f"sum_qtd_digital_u{months}m"),
        # # pct_pre_pago
        # F.sum("pct_pre_pago").over(w).alias(f"sum_pct_pre_pago_u{months}m"),
        # # pct_flex
        # F.sum("pct_flex").over(w).alias(f"sum_pct_flex_u{months}m"),
        # # pct_autocontrole
        # F.sum("pct_autocontrole").over(w).alias(f"sum_pct_autocontrole_u{months}m"),
        # # pct_digital
        # F.sum("pct_digital").over(w).alias(f"sum_pct_digital_u{months}m"),
        # # pct_flex_lag
        # F.sum("pct_flex_lag").over(w).alias(f"sum_pct_flex_lag_u{months}m"),
        # # pct_linha_ativa
        # F.sum("pct_linha_ativa").over(w).alias(f"sum_pct_linha_ativa_u{months}m"),
        # # pct_expirado_recente
        # F.sum("pct_expirado_recente").over(w).alias(f"sum_pct_expirado_recente_u{months}m"),
        # # QTD_RECARGAS_DIGITAL
        # F.sum("QTD_RECARGAS_DIGITAL").over(w).alias(f"sum_QTD_RECARGAS_DIGITAL_u{months}m"),
        # # QTD_RECARGAS_BANCO
        # F.sum("QTD_RECARGAS_BANCO").over(w).alias(f"sum_QTD_RECARGAS_BANCO_u{months}m"),
        # # QTD_RECARGAS_VAREJO
        # F.sum("QTD_RECARGAS_VAREJO").over(w).alias(f"sum_QTD_RECARGAS_VAREJO_u{months}m"),
        # # QTD_RECARGAS_TELECOM
        # F.sum("QTD_RECARGAS_TELECOM").over(w).alias(f"sum_QTD_RECARGAS_TELECOM_u{months}m"),
        # # PCT_RECARGA_DIGITAL
        # F.sum("PCT_RECARGA_DIGITAL").over(w).alias(f"sum_PCT_RECARGA_DIGITAL_u{months}m"),
        # # PCT_RECARGA_BANCO
        # F.sum("PCT_RECARGA_BANCO").over(w).alias(f"sum_PCT_RECARGA_BANCO_u{months}m"),
        # # PCT_RECARGA_VAREJO
        # F.sum("PCT_RECARGA_VAREJO").over(w).alias(f"sum_PCT_RECARGA_VAREJO_u{months}m"),
        # # PCT_RECARGA_TELECOM
        # F.sum("PCT_RECARGA_TELECOM").over(w).alias(f"sum_PCT_RECARGA_TELECOM_u{months}m"),
        # # PCT_GRUPO_CARTAO_DOMINANTE
        # F.sum("PCT_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"sum_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # # QTD_GRUPOS_CARTAO_DISTINTOS
        # F.sum("QTD_GRUPOS_CARTAO_DISTINTOS").over(w).alias(f"sum_QTD_GRUPOS_CARTAO_DISTINTOS_u{months}m"),
        # # PCT_DSC_GRUPO_CARTAO_DOMINANTE
        # F.sum("PCT_DSC_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"sum_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # # QTD_DSC_GRUPO_CARTAO_DISTINTOS
        # F.sum("QTD_DSC_GRUPO_CARTAO_DISTINTOS").over(w).alias(f"sum_QTD_DSC_GRUPO_CARTAO_DISTINTOS_u{months}m"),
        # # recencia_dias
        # F.sum("recencia_dias").over(w).alias(f"sum_recencia_dias_u{months}m"),


        # =========================
        # EXTREMOS/MÍNIMO MÁXIMO
        # =========================
        #  F.min("vlr_medio_recarga").over(w).alias(f"min_vlr_medio_recarga_u{months}m"),
        # F.max("vlr_medio_recarga").over(w).alias(f"max_vlr_medio_recarga_u{months}m"),
        # F.min("vlr_max_recarga").over(w).alias(f"min_vlr_max_recarga_u{months}m"),
        # F.max("vlr_max_recarga").over(w).alias(f"max_vlr_max_recarga_u{months}m"),
        # F.min("vlr_min_recarga").over(w).alias(f"min_vlr_min_recarga_u{months}m"),
        # F.max("vlr_min_recarga").over(w).alias(f"max_vlr_min_recarga_u{months}m"),
        # F.min("vlr_total_recarga").over(w).alias(f"min_vlr_total_recarga_u{months}m"),
        # F.max("vlr_total_recarga").over(w).alias(f"max_vlr_total_recarga_u{months}m"),
        # F.min("vlr_total_bonus").over(w).alias(f"min_vlr_total_bonus_u{months}m"),
        #  F.max("vlr_total_bonus").over(w).alias(f"max_vlr_total_bonus_u{months}m"),
        # F.min("vlr_medio_bonus").over(w).alias(f"min_vlr_medio_bonus_u{months}m"),
        # F.max("vlr_medio_bonus").over(w).alias(f"max_vlr_medio_bonus_u{months}m"),
        # F.min("vlr_total_real").over(w).alias(f"min_vlr_total_real_u{months}m"),
        # F.max("vlr_total_real").over(w).alias(f"max_vlr_total_real_u{months}m"),
        # F.min("vlr_medio_real").over(w).alias(f"min_vlr_medio_real_u{months}m"),
        # F.max("vlr_medio_real").over(w).alias(f"max_vlr_medio_real_u{months}m"),
        # F.min("vlr_medio_rec_sos").over(w).alias(f"min_vlr_medio_rec_sos_u{months}m"),
        # F.max("vlr_medio_rec_sos").over(w).alias(f"max_vlr_medio_rec_sos_u{months}m"),
        # F.min("pct_rec_sos").over(w).alias(f"min_pct_rec_sos_u{months}m"),
        #  F.max("pct_rec_sos").over(w).alias(f"max_pct_rec_sos_u{months}m"),
        # F.min("vlr_rec_com_bonus").over(w).alias(f"min_vlr_rec_com_bonus_u{months}m"),
        # F.max("vlr_rec_com_bonus").over(w).alias(f"max_vlr_rec_com_bonus_u{months}m"),
        # F.min("pct_rec_com_bonus").over(w).alias(f"min_pct_rec_com_bonus_u{months}m"),
        # F.max("pct_rec_com_bonus").over(w).alias(f"max_pct_rec_com_bonus_u{months}m"),
        # F.min("gap_medio_recarga").over(w).alias(f"min_gap_medio_recarga_u{months}m"),
        #  F.max("gap_medio_recarga").over(w).alias(f"max_gap_medio_recarga_u{months}m"),
        # F.min("gap_std_recarga").over(w).alias(f"min_gap_std_recarga_u{months}m"),
        #  F.max("gap_std_recarga").over(w).alias(f"max_gap_std_recarga_u{months}m"),
        # F.min("desvio_recencia").over(w).alias(f"min_desvio_recencia_u{months}m"),
        # F.max("desvio_recencia").over(w).alias(f"max_desvio_recencia_u{months}m"),
        # F.min("pct_recarga_adicional_controle").over(w).alias(f"min_pct_recarga_adicional_controle_u{months}m"),
        # F.max("pct_recarga_adicional_controle").over(w).alias(f"max_pct_recarga_adicional_controle_u{months}m"),
        # F.min("pct_rec_digital").over(w).alias(f"min_pct_rec_digital_u{months}m"),
        # F.max("pct_rec_digital").over(w).alias(f"max_pct_rec_digital_u{months}m"),
        # F.min("pct_rec_varejo").over(w).alias(f"min_pct_rec_varejo_u{months}m"),
        # F.max("pct_rec_varejo").over(w).alias(f"max_pct_rec_varejo_u{months}m"),
        # F.min("pct_pre_pago").over(w).alias(f"min_pct_pre_pago_u{months}m"),
        # F.max("pct_pre_pago").over(w).alias(f"max_pct_pre_pago_u{months}m"),
        # F.min("pct_flex").over(w).alias(f"min_pct_flex_u{months}m"),
        # F.max("pct_flex").over(w).alias(f"max_pct_flex_u{months}m"),
        # F.min("pct_autocontrole").over(w).alias(f"min_pct_autocontrole_u{months}m"),
        # F.max("pct_autocontrole").over(w).alias(f"max_pct_autocontrole_u{months}m"),
        # F.min("pct_digital").over(w).alias(f"min_pct_digital_u{months}m"),
        # F.max("pct_digital").over(w).alias(f"max_pct_digital_u{months}m"),
        # F.min("pct_flex_lag").over(w).alias(f"min_pct_flex_lag_u{months}m"),
        # F.max("pct_flex_lag").over(w).alias(f"max_pct_flex_lag_u{months}m"),
        # F.min("pct_linha_ativa").over(w).alias(f"min_pct_linha_ativa_u{months}m"),
        # F.max("pct_linha_ativa").over(w).alias(f"max_pct_linha_ativa_u{months}m"),
        # F.min("pct_expirado_recente").over(w).alias(f"min_pct_expirado_recente_u{months}m"),
        # F.max("pct_expirado_recente").over(w).alias(f"max_pct_expirado_recente_u{months}m"),
        # F.min("QTD_RECARGAS_DIGITAL").over(w).alias(f"min_QTD_RECARGAS_DIGITAL_u{months}m"),
        # F.max("QTD_RECARGAS_DIGITAL").over(w).alias(f"max_QTD_RECARGAS_DIGITAL_u{months}m"),
        # F.min("QTD_RECARGAS_BANCO").over(w).alias(f"min_QTD_RECARGAS_BANCO_u{months}m"),
        # F.max("QTD_RECARGAS_BANCO").over(w).alias(f"max_QTD_RECARGAS_BANCO_u{months}m"),
        # F.min("QTD_RECARGAS_VAREJO").over(w).alias(f"min_QTD_RECARGAS_VAREJO_u{months}m"),
        # F.max("QTD_RECARGAS_VAREJO").over(w).alias(f"max_QTD_RECARGAS_VAREJO_u{months}m"),
        # F.min("QTD_RECARGAS_TELECOM").over(w).alias(f"min_QTD_RECARGAS_TELECOM_u{months}m"),
        # F.max("QTD_RECARGAS_TELECOM").over(w).alias(f"max_QTD_RECARGAS_TELECOM_u{months}m"),
        # F.min("PCT_RECARGA_DIGITAL").over(w).alias(f"min_PCT_RECARGA_DIGITAL_u{months}m"),
        # F.max("PCT_RECARGA_DIGITAL").over(w).alias(f"max_PCT_RECARGA_DIGITAL_u{months}m"),
        # F.min("PCT_RECARGA_BANCO").over(w).alias(f"min_PCT_RECARGA_BANCO_u{months}m"),
        # F.max("PCT_RECARGA_BANCO").over(w).alias(f"max_PCT_RECARGA_BANCO_u{months}m"),
        # F.min("PCT_RECARGA_VAREJO").over(w).alias(f"min_PCT_RECARGA_VAREJO_u{months}m"),
        # F.max("PCT_RECARGA_VAREJO").over(w).alias(f"max_PCT_RECARGA_VAREJO_u{months}m"),
        # F.min("PCT_RECARGA_TELECOM").over(w).alias(f"min_PCT_RECARGA_TELECOM_u{months}m"),
        # F.max("PCT_RECARGA_TELECOM").over(w).alias(f"max_PCT_RECARGA_TELECOM_u{months}m"),
        # F.min("PCT_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"min_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # F.max("PCT_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"max_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # F.min("QTD_GRUPOS_CARTAO_DISTINTOS").over(w).alias(f"min_QTD_GRUPOS_CARTAO_DISTINTOS_u{months}m"),
        # F.max("QTD_GRUPOS_CARTAO_DISTINTOS").over(w).alias(f"max_QTD_GRUPOS_CARTAO_DISTINTOS_u{months}m"),
        # F.min("PCT_DSC_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"min_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # F.max("PCT_DSC_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"max_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
        # F.min("QTD_DSC_GRUPO_CARTAO_DISTINTOS").over(w).alias(f"min_QTD_DSC_GRUPO_CARTAO_DISTINTOS_u{months}m"),
        # F.max("QTD_DSC_GRUPO_CARTAO_DISTINTOS").over(w).alias(f"max_QTD_DSC_GRUPO_CARTAO_DISTINTOS_u{months}m"),
        #  F.max("qtd_recargas").over(w).alias(f"max_qtd_recargas_u{months}m"),
        # F.min("qtd_recargas").over(w).alias(f"min_qtd_recargas_u{months}m"),
        # F.max("qtd_telefones").over(w).alias(f"max_qtd_telefones_u{months}m"),
        # F.min("qtd_telefones").over(w).alias(f"min_qtd_telefones_u{months}m"),
        # F.max("qtd_rec_madrugada").over(w).alias(f"max_qtd_rec_madrugada_u{months}m"),
        # F.min("qtd_rec_madrugada").over(w).alias(f"min_qtd_rec_madrugada_u{months}m"),
        # F.max("qtd_rec_manha").over(w).alias(f"max_qtd_rec_manha_u{months}m"),
        # F.min("qtd_rec_manha").over(w).alias(f"min_qtd_rec_manha_u{months}m"),
        # F.max("qtd_rec_tarde").over(w).alias(f"max_qtd_rec_tarde_u{months}m"),
        # F.min("qtd_rec_tarde").over(w).alias(f"min_qtd_rec_tarde_u{months}m"),
        # F.max("qtd_rec_noite").over(w).alias(f"max_qtd_rec_noite_u{months}m"),
        # F.min("qtd_rec_noite").over(w).alias(f"min_qtd_rec_noite_u{months}m"),
        #  F.max("qtd_rec_sos").over(w).alias(f"max_qtd_rec_sos_u{months}m"),
        # F.min("qtd_rec_sos").over(w).alias(f"min_qtd_rec_sos_u{months}m"),
        # F.max("qtd_rec_com_bonus").over(w).alias(f"max_qtd_rec_com_bonus_u{months}m"),
        # F.min("qtd_rec_com_bonus").over(w).alias(f"min_qtd_rec_com_bonus_u{months}m"),
        #  F.max("recencia_dias").over(w).alias(f"max_recencia_dias_u{months}m"),
        #  F.min("recencia_dias").over(w).alias(f"min_recencia_dias_u{months}m"),
        # F.max("score_risco_recencia").over(w).alias(f"max_score_risco_recencia_u{months}m"),
        # F.min("score_risco_recencia").over(w).alias(f"min_score_risco_recencia_u{months}m"),
        # F.max("qtd_rec_plano_541200").over(w).alias(f"max_qtd_rec_plano_541200_u{months}m"),
        # F.min("qtd_rec_plano_541200").over(w).alias(f"min_qtd_rec_plano_541200_u{months}m"),
        # F.max("qtd_rec_plano_668900").over(w).alias(f"max_qtd_rec_plano_668900_u{months}m"),
        # F.min("qtd_rec_plano_668900").over(w).alias(f"min_qtd_rec_plano_668900_u{months}m"),
        # F.max("qtd_rec_plano_599100").over(w).alias(f"max_qtd_rec_plano_599100_u{months}m"),
        # F.min("qtd_rec_plano_599100").over(w).alias(f"min_qtd_rec_plano_599100_u{months}m"),
        # F.max("qtd_rec_plano_607690").over(w).alias(f"max_qtd_rec_plano_607690_u{months}m"),
        # F.min("qtd_rec_plano_607690").over(w).alias(f"min_qtd_rec_plano_607690_u{months}m"),
        # F.max("qtd_rec_plano_999999").over(w).alias(f"max_qtd_rec_plano_999999_u{months}m"),
        # F.min("qtd_rec_plano_999999").over(w).alias(f"min_qtd_rec_plano_999999_u{months}m"),
        # F.max("qtd_rec_plano_369889").over(w).alias(f"max_qtd_rec_plano_369889_u{months}m"),
        # F.min("qtd_rec_plano_369889").over(w).alias(f"min_qtd_rec_plano_369889_u{months}m"),
        # F.max("qtd_rec_plano_393401").over(w).alias(f"max_qtd_rec_plano_393401_u{months}m"),
        # F.min("qtd_rec_plano_393401").over(w).alias(f"min_qtd_rec_plano_393401_u{months}m"),
        # F.max("qtd_tipo_insercao").over(w).alias(f"max_qtd_tipo_insercao_u{months}m"),
        # F.min("qtd_tipo_insercao").over(w).alias(f"min_qtd_tipo_insercao_u{months}m"),
        # F.max("qtd_forma_pgto_na").over(w).alias(f"max_qtd_tipo_recarga_u{months}m"),
        # F.min("qtd_forma_pgto_na").over(w).alias(f"min_qtd_tipo_recarga_u{months}m"),
        # F.max("qtd_forma_pgto_nd").over(w).alias(f"max_qtd_forma_pgto_nd_u{months}m"),
        # F.min("qtd_forma_pgto_nd").over(w).alias(f"min_qtd_forma_pgto_nd_u{months}m"),
        # F.max("qtd_forma_pgto_ni").over(w).alias(f"max_qtd_forma_pgto_ni_u{months}m"),
        # F.min("qtd_forma_pgto_ni").over(w).alias(f"min_qtd_forma_pgto_ni_u{months}m"),
        # F.max("qtd_forma_pgto_cp").over(w).alias(f"max_qtd_forma_pgto_cp_u{months}m"),
        # F.min("qtd_forma_pgto_cp").over(w).alias(f"min_qtd_forma_pgto_cp_u{months}m"),
        # F.max("qtd_forma_pgto_dd").over(w).alias(f"max_qtd_forma_pgto_dd_u{months}m"),
        # F.min("qtd_forma_pgto_dd").over(w).alias(f"min_qtd_forma_pgto_dd_u{months}m"),
        # F.max("qtd_forma_pgto_mp").over(w).alias(f"max_qtd_forma_pgto_mp_u{months}m"),
        # F.min("qtd_forma_pgto_mp").over(w).alias(f"min_qtd_forma_pgto_mp_u{months}m"),
        # F.max("qtd_forma_pgto_pb").over(w).alias(f"max_qtd_forma_pgto_pb_u{months}m"),
        # F.min("qtd_forma_pgto_pb").over(w).alias(f"min_qtd_forma_pgto_pb_u{months}m"),
        # F.max("qtd_forma_pgto_ca").over(w).alias(f"max_qtd_forma_pgto_ca_u{months}m"),
        # F.min("qtd_forma_pgto_ca").over(w).alias(f"min_qtd_forma_pgto_ca_u{months}m"),
        # F.max("qtd_forma_pgto_pa").over(w).alias(f"max_qtd_forma_pgto_pa_u{months}m"),
        # F.min("qtd_forma_pgto_pa").over(w).alias(f"min_qtd_forma_pgto_pa_u{months}m"),
        # F.max("qtd_recarga_adicional_controle").over(w).alias(f"max_qtd_recarga_adicional_controle_u{months}m"),
        # F.min("qtd_recarga_adicional_controle").over(w).alias(f"min_qtd_recarga_adicional_controle_u{months}m"),
        # F.max("qtd_rec_promocao").over(w).alias(f"max_qtd_rec_promocao_u{months}m"),
        # F.min("qtd_rec_promocao").over(w).alias(f"min_qtd_rec_promocao_u{months}m"),
        #  F.max("qtd_canais_distintos").over(w).alias(f"max_qtd_canais_distintos_u{months}m"),
        # F.min("qtd_canais_distintos").over(w).alias(f"min_qtd_canais_distintos_u{months}m"),
        # F.max("qtd_eventos").over(w).alias(f"max_qtd_eventos_u{months}m"),
        # F.min("qtd_eventos").over(w).alias(f"min_qtd_eventos_u{months}m"),
        # F.max("qtd_pre_pago").over(w).alias(f"max_qtd_pre_pago_u{months}m"),
        # F.min("qtd_pre_pago").over(w).alias(f"min_qtd_pre_pago_u{months}m"),
        # F.max("qtd_flex").over(w).alias(f"max_qtd_flex_u{months}m"),
        # F.min("qtd_flex").over(w).alias(f"min_qtd_flex_u{months}m"),
        # F.max("qtd_autocontrole").over(w).alias(f"max_qtd_autocontrole_u{months}m"),
        # F.min("qtd_autocontrole").over(w).alias(f"min_qtd_autocontrole_u{months}m"),
        # F.max("qtd_digital").over(w).alias(f"max_qtd_digital_u{months}m"),
        # F.min("qtd_digital").over(w).alias(f"min_qtd_digital_u{months}m"),
        # F.max("qtd_plataformas_distintas").over(w).alias(f"max_qtd_plataformas_distintas_u{months}m"),
        # F.min("qtd_plataformas_distintas").over(w).alias(f"min_qtd_plataformas_distintas_u{months}m"),
        # F.max("qtd_status_distintos").over(w).alias(f"max_qtd_status_distintos_u{months}m"),
        # F.min("qtd_status_distintos").over(w).alias(f"min_qtd_status_distintos_u{months}m"),


        # =========================
        # DISTRIBUIÇÃO (cauda)
        # =========================
        # F.skewness("vlr_total_recarga").over(w).alias(f"skewness_vlr_total_recarga_u{months}m"),
        #  F.skewness("vlr_medio_recarga").over(w).alias(f"skewness_vlr_medio_recarga_u{months}m"),
        # F.skewness("vlr_max_recarga").over(w).alias(f"skewness_vlr_max_recarga_u{months}m"),
        # F.skewness("vlr_min_recarga").over(w).alias(f"skewness_vlr_min_recarga_u{months}m"),
        #  F.skewness("vlr_total_bonus").over(w).alias(f"skewness_vlr_total_bonus_u{months}m"),
        # F.skewness("vlr_medio_bonus").over(w).alias(f"skewness_vlr_medio_bonus_u{months}m"),
        # F.skewness("vlr_total_real").over(w).alias(f"skewness_vlr_total_real_u{months}m"),
        # F.skewness("vlr_medio_real").over(w).alias(f"skewness_vlr_medio_real_u{months}m"),
        # F.skewness("vlr_medio_rec_sos").over(w).alias(f"skewness_vlr_medio_rec_sos_u{months}m"),
        #  F.skewness("pct_rec_sos").over(w).alias(f"skewness_pct_rec_sos_u{months}m"),
        # F.skewness("vlr_rec_com_bonus").over(w).alias(f"skewness_vlr_rec_com_bonus_u{months}m"),
        # F.skewness("pct_rec_com_bonus").over(w).alias(f"skewness_pct_rec_com_bonus_u{months}m"),
        #  F.skewness("gap_medio_recarga").over(w).alias(f"skewness_gap_medio_recarga_u{months}m"),
        # F.skewness("gap_std_recarga").over(w).alias(f"skewness_gap_std_recarga_u{months}m"),
        #  F.skewness("desvio_recencia").over(w).alias(f"skewness_desvio_recencia_u{months}m"),
        # F.skewness("score_risco_recencia").over(w).alias(f"skewness_score_risco_recencia_u{months}m"),
        # F.skewness("pct_recarga_adicional_controle").over(w).alias(f"skewness_pct_recarga_adicional_controle_u{months}m"),
        # F.skewness("pct_rec_digital").over(w).alias(f"skewness_pct_rec_digital_u{months}m"),
        # F.skewness("pct_rec_varejo").over(w).alias(f"skewness_pct_rec_varejo_u{months}m"),
        # F.skewness("pct_pre_pago").over(w).alias(f"skewness_pct_pre_pago_u{months}m"),
        # F.skewness("pct_flex").over(w).alias(f"skewness_pct_flex_u{months}m"),
        # F.skewness("pct_autocontrole").over(w).alias(f"skewness_pct_autocontrole_u{months}m"),
        # F.skewness("pct_digital").over(w).alias(f"skewness_pct_digital_u{months}m"),
        # F.skewness("pct_flex_lag").over(w).alias(f"skewness_pct_flex_lag_u{months}m"),
        # F.skewness("pct_linha_ativa").over(w).alias(f"skewness_pct_linha_ativa_u{months}m"),
        # F.skewness("pct_expirado_recente").over(w).alias(f"skewness_pct_expirado_recente_u{months}m"),
        # F.skewness("QTD_RECARGAS_DIGITAL").over(w).alias(f"skewness_QTD_RECARGAS_DIGITAL_u{months}m"),
        # F.skewness("QTD_RECARGAS_BANCO").over(w).alias(f"skewness_QTD_RECARGAS_BANCO_u{months}m"),
        # F.skewness("QTD_RECARGAS_VAREJO").over(w).alias(f"skewness_QTD_RECARGAS_VAREJO_u{months}m"),
        # F.skewness("QTD_RECARGAS_TELECOM").over(w).alias(f"skewness_QTD_RECARGAS_TELECOM_u{months}m"),
        # F.skewness("PCT_RECARGA_DIGITAL").over(w).alias(f"skewness_PCT_RECARGA_DIGITAL_u{months}m"),
        # F.skewness("PCT_RECARGA_BANCO").over(w).alias(f"skewness_PCT_RECARGA_BANCO_u{months}m"),
        # F.skewness("PCT_RECARGA_VAREJO").over(w).alias(f"skewness_PCT_RECARGA_VAREJO_u{months}m"),
        # F.skewness("PCT_RECARGA_TELECOM").over(w).alias(f"skewness_PCT_RECARGA_TELECOM_u{months}m"),
        # F.skewness("PCT_GRUPO_CARTAO_DOMINANTE").over(w).alias(f"skewness_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),

            # =========================
            # QUARTIZ
            # =========================


            # =========================
            # PERCENTIL
            # # =========================
            # F.percentile_approx("vlr_total_recarga", 0.25).over(w).alias(f"p25_vlr_total_recarga_u{months}m"),
            # F.percentile_approx("vlr_total_recarga", 0.50).over(w).alias(f"p50_vlr_total_recarga_u{months}m"),
            # F.percentile_approx("vlr_total_recarga", 0.75).over(w).alias(f"p75_vlr_total_recarga_u{months}m"),
            #  F.percentile_approx("vlr_total_recarga", 0.90).over(w).alias(f"p90_vlr_total_recarga_u{months}m"),
            # F.percentile_approx("vlr_total_recarga", 0.95).over(w).alias(f"p95_vlr_total_recarga_u{months}m"),
            # F.percentile_approx("vlr_total_recarga", 0.99).over(w).alias(f"p99_vlr_total_recarga_u{months}m"),
            # F.percentile_approx("vlr_medio_recarga", 0.25).over(w).alias(f"p25_vlr_medio_recarga_u{months}m"),
            #  F.percentile_approx("vlr_medio_recarga", 0.50).over(w).alias(f"p50_vlr_medio_recarga_u{months}m"),
            # F.percentile_approx("vlr_medio_recarga", 0.75).over(w).alias(f"p75_vlr_medio_recarga_u{months}m"),
            # F.percentile_approx("vlr_medio_recarga", 0.90).over(w).alias(f"p90_vlr_medio_recarga_u{months}m"),
            # F.percentile_approx("vlr_medio_recarga", 0.95).over(w).alias(f"p95_vlr_medio_recarga_u{months}m"),
            # F.percentile_approx("vlr_medio_recarga", 0.99).over(w).alias(f"p99_vlr_medio_recarga_u{months}m"),
            # F.percentile_approx("vlr_max_recarga", 0.25).over(w).alias(f"p25_vlr_max_recarga_u{months}m"),
            # F.percentile_approx("vlr_max_recarga", 0.50).over(w).alias(f"p50_vlr_max_recarga_u{months}m"),
            # F.percentile_approx("vlr_max_recarga", 0.75).over(w).alias(f"p75_vlr_max_recarga_u{months}m"),
            # F.percentile_approx("vlr_max_recarga", 0.90).over(w).alias(f"p90_vlr_max_recarga_u{months}m"),
            # F.percentile_approx("vlr_max_recarga", 0.95).over(w).alias(f"p95_vlr_max_recarga_u{months}m"),
            # F.percentile_approx("vlr_max_recarga", 0.99).over(w).alias(f"p99_vlr_max_recarga_u{months}m"),
            # F.percentile_approx("vlr_min_recarga", 0.25).over(w).alias(f"p25_vlr_min_recarga_u{months}m"),
            # F.percentile_approx("vlr_min_recarga", 0.50).over(w).alias(f"p50_vlr_min_recarga_u{months}m"),
            # F.percentile_approx("vlr_min_recarga", 0.75).over(w).alias(f"p75_vlr_min_recarga_u{months}m"),
            # F.percentile_approx("vlr_min_recarga", 0.90).over(w).alias(f"p90_vlr_min_recarga_u{months}m"),
            # F.percentile_approx("vlr_min_recarga", 0.95).over(w).alias(f"p95_vlr_min_recarga_u{months}m"),
            # F.percentile_approx("vlr_min_recarga", 0.99).over(w).alias(f"p99_vlr_min_recarga_u{months}m"),
            # F.percentile_approx("vlr_total_bonus", 0.25).over(w).alias(f"p25_vlr_total_bonus_u{months}m"),
            # F.percentile_approx("vlr_total_bonus", 0.50).over(w).alias(f"p50_vlr_total_bonus_u{months}m"),
            # F.percentile_approx("vlr_total_bonus", 0.75).over(w).alias(f"p75_vlr_total_bonus_u{months}m"),
            # F.percentile_approx("vlr_total_bonus", 0.90).over(w).alias(f"p90_vlr_total_bonus_u{months}m"),
            # F.percentile_approx("vlr_total_bonus", 0.95).over(w).alias(f"p95_vlr_total_bonus_u{months}m"),
            # F.percentile_approx("vlr_total_bonus", 0.99).over(w).alias(f"p99_vlr_total_bonus_u{months}m"),
            # F.percentile_approx("vlr_medio_bonus", 0.25).over(w).alias(f"p25_vlr_medio_bonus_u{months}m"),
            # F.percentile_approx("vlr_medio_bonus", 0.50).over(w).alias(f"p50_vlr_medio_bonus_u{months}m"),
            # F.percentile_approx("vlr_medio_bonus", 0.75).over(w).alias(f"p75_vlr_medio_bonus_u{months}m"),
            # F.percentile_approx("vlr_medio_bonus", 0.90).over(w).alias(f"p90_vlr_medio_bonus_u{months}m"),
            # F.percentile_approx("vlr_medio_bonus", 0.95).over(w).alias(f"p95_vlr_medio_bonus_u{months}m"),
            # F.percentile_approx("vlr_medio_bonus", 0.99).over(w).alias(f"p99_vlr_medio_bonus_u{months}m"),
            # F.percentile_approx("vlr_total_real", 0.25).over(w).alias(f"p25_vlr_total_real_u{months}m"),
            #  F.percentile_approx("vlr_total_real", 0.50).over(w).alias(f"p50_vlr_total_real_u{months}m"),
            # F.percentile_approx("vlr_total_real", 0.75).over(w).alias(f"p75_vlr_total_real_u{months}m"),
            # F.percentile_approx("vlr_total_real", 0.90).over(w).alias(f"p90_vlr_total_real_u{months}m"),
            # F.percentile_approx("vlr_total_real", 0.95).over(w).alias(f"p95_vlr_total_real_u{months}m"),
            # F.percentile_approx("vlr_total_real", 0.99).over(w).alias(f"p99_vlr_total_real_u{months}m"),
            # F.percentile_approx("vlr_medio_real", 0.25).over(w).alias(f"p25_vlr_medio_real_u{months}m"),
            # F.percentile_approx("vlr_medio_real", 0.50).over(w).alias(f"p50_vlr_medio_real_u{months}m"),
            # F.percentile_approx("vlr_medio_real", 0.75).over(w).alias(f"p75_vlr_medio_real_u{months}m"),
            # F.percentile_approx("vlr_medio_real", 0.90).over(w).alias(f"p90_vlr_medio_real_u{months}m"),
            # F.percentile_approx("vlr_medio_real", 0.95).over(w).alias(f"p95_vlr_medio_real_u{months}m"),
            # F.percentile_approx("vlr_medio_real", 0.99).over(w).alias(f"p99_vlr_medio_real_u{months}m"),
            # F.percentile_approx("vlr_medio_rec_sos", 0.25).over(w).alias(f"p25_vlr_medio_rec_sos_u{months}m"),
            # F.percentile_approx("vlr_medio_rec_sos", 0.50).over(w).alias(f"p50_vlr_medio_rec_sos_u{months}m"),
            # F.percentile_approx("vlr_medio_rec_sos", 0.75).over(w).alias(f"p75_vlr_medio_rec_sos_u{months}m"),
            # F.percentile_approx("vlr_medio_rec_sos", 0.90).over(w).alias(f"p90_vlr_medio_rec_sos_u{months}m"),
            # F.percentile_approx("vlr_medio_rec_sos", 0.95).over(w).alias(f"p95_vlr_medio_rec_sos_u{months}m"),
            # F.percentile_approx("vlr_medio_rec_sos", 0.99).over(w).alias(f"p99_vlr_medio_rec_sos_u{months}m"),
            # F.percentile_approx("pct_rec_sos", 0.25).over(w).alias(f"p25_pct_rec_sos_u{months}m"),
            # F.percentile_approx("pct_rec_sos", 0.50).over(w).alias(f"p50_pct_rec_sos_u{months}m"),
            #  F.percentile_approx("pct_rec_sos", 0.75).over(w).alias(f"p75_pct_rec_sos_u{months}m"),
            # F.percentile_approx("pct_rec_sos", 0.90).over(w).alias(f"p90_pct_rec_sos_u{months}m"),
            # F.percentile_approx("pct_rec_sos", 0.95).over(w).alias(f"p95_pct_rec_sos_u{months}m"),
            # F.percentile_approx("pct_rec_sos", 0.99).over(w).alias(f"p99_pct_rec_sos_u{months}m"),

           # F.percentile_approx("recencia_dias", 0.25).over(w).alias(f"p25_pct_recencia_dias_u{months}m"),
            #  F.percentile_approx("recencia_dias", 0.50).over(w).alias(f"p50_pct_recencia_dias_u{months}m"),
            # F.percentile_approx("recencia_dias", 0.75).over(w).alias(f"p75_pct_recencia_dias_u{months}m"),
            #  F.percentile_approx("recencia_dias", 0.90).over(w).alias(f"p90_pct_recencia_dias_u{months}m"),
            # F.percentile_approx("recencia_dias", 0.95).over(w).alias(f"p95_pct_recencia_dias_u{months}m"),
            # F.percentile_approx("recencia_dias", 0.99).over(w).alias(f"p99_pct_recencia_dias_u{months}m"),

            # F.percentile_approx("vlr_rec_com_bonus", 0.25).over(w).alias(f"p25_vlr_rec_com_bonus_u{months}m"),
            # F.percentile_approx("vlr_rec_com_bonus", 0.50).over(w).alias(f"p50_vlr_rec_com_bonus_u{months}m"),
            #  F.percentile_approx("vlr_rec_com_bonus", 0.75).over(w).alias(f"p75_vlr_rec_com_bonus_u{months}m"),
            # F.percentile_approx("vlr_rec_com_bonus", 0.90).over(w).alias(f"p90_vlr_rec_com_bonus_u{months}m"),
            # F.percentile_approx("vlr_rec_com_bonus", 0.95).over(w).alias(f"p95_vlr_rec_com_bonus_u{months}m"),
            # F.percentile_approx("vlr_rec_com_bonus", 0.99).over(w).alias(f"p99_vlr_rec_com_bonus_u{months}m"),
            # F.percentile_approx("pct_rec_com_bonus", 0.25).over(w).alias(f"p25_pct_rec_com_bonus_u{months}m"),
            # F.percentile_approx("pct_rec_com_bonus", 0.50).over(w).alias(f"p50_pct_rec_com_bonus_u{months}m"),
            # F.percentile_approx("pct_rec_com_bonus", 0.75).over(w).alias(f"p75_pct_rec_com_bonus_u{months}m"),
            # F.percentile_approx("pct_rec_com_bonus", 0.90).over(w).alias(f"p90_pct_rec_com_bonus_u{months}m"),
            # F.percentile_approx("pct_rec_com_bonus", 0.95).over(w).alias(f"p95_pct_rec_com_bonus_u{months}m"),
            # F.percentile_approx("pct_rec_com_bonus", 0.99).over(w).alias(f"p99_pct_rec_com_bonus_u{months}m"),
            # F.percentile_approx("gap_medio_recarga", 0.25).over(w).alias(f"p25_gap_medio_recarga_u{months}m"),
            #  F.percentile_approx("gap_medio_recarga", 0.50).over(w).alias(f"p50_gap_medio_recarga_u{months}m"),
            # F.percentile_approx("gap_medio_recarga", 0.75).over(w).alias(f"p75_gap_medio_recarga_u{months}m"),
            #  F.percentile_approx("gap_medio_recarga", 0.90).over(w).alias(f"p90_gap_medio_recarga_u{months}m"),
            # F.percentile_approx("gap_medio_recarga", 0.95).over(w).alias(f"p95_gap_medio_recarga_u{months}m"),
            # F.percentile_approx("gap_medio_recarga", 0.99).over(w).alias(f"p99_gap_medio_recarga_u{months}m"),
            # F.percentile_approx("gap_std_recarga", 0.25).over(w).alias(f"p25_gap_std_recarga_u{months}m"),
            # F.percentile_approx("gap_std_recarga", 0.50).over(w).alias(f"p50_gap_std_recarga_u{months}m"),
            #  F.percentile_approx("gap_std_recarga", 0.75).over(w).alias(f"p75_gap_std_recarga_u{months}m"),
            # F.percentile_approx("gap_std_recarga", 0.90).over(w).alias(f"p90_gap_std_recarga_u{months}m"),
            # F.percentile_approx("gap_std_recarga", 0.95).over(w).alias(f"p95_gap_std_recarga_u{months}m"),
            # F.percentile_approx("gap_std_recarga", 0.99).over(w).alias(f"p99_gap_std_recarga_u{months}m"),
            # F.percentile_approx("desvio_recencia", 0.25).over(w).alias(f"p25_desvio_recencia_u{months}m"),
            # F.percentile_approx("desvio_recencia", 0.50).over(w).alias(f"p50_desvio_recencia_u{months}m"),
            # F.percentile_approx("desvio_recencia", 0.75).over(w).alias(f"p75_desvio_recencia_u{months}m"),
            # F.percentile_approx("desvio_recencia", 0.90).over(w).alias(f"p90_desvio_recencia_u{months}m"),
            # F.percentile_approx("desvio_recencia", 0.95).over(w).alias(f"p95_desvio_recencia_u{months}m"),
            # F.percentile_approx("desvio_recencia", 0.99).over(w).alias(f"p99_desvio_recencia_u{months}m"),
            # F.percentile_approx("score_risco_recencia", 0.25).over(w).alias(f"p25_score_risco_recencia_u{months}m"),
            # F.percentile_approx("score_risco_recencia", 0.50).over(w).alias(f"p50_score_risco_recencia_u{months}m"),
            # F.percentile_approx("score_risco_recencia", 0.75).over(w).alias(f"p75_score_risco_recencia_u{months}m"),
            # F.percentile_approx("score_risco_recencia", 0.90).over(w).alias(f"p90_score_risco_recencia_u{months}m"),
            # F.percentile_approx("score_risco_recencia", 0.95).over(w).alias(f"p95_score_risco_recencia_u{months}m"),
            # F.percentile_approx("score_risco_recencia", 0.99).over(w).alias(f"p99_score_risco_recencia_u{months}m"),
            # F.percentile_approx("pct_recarga_adicional_controle", 0.25).over(w).alias(f"p25_pct_recarga_adicional_controle_u{months}m"),
            # F.percentile_approx("pct_recarga_adicional_controle", 0.50).over(w).alias(f"p50_pct_recarga_adicional_controle_u{months}m"),
            # F.percentile_approx("pct_recarga_adicional_controle", 0.75).over(w).alias(f"p75_pct_recarga_adicional_controle_u{months}m"),
            # F.percentile_approx("pct_recarga_adicional_controle", 0.90).over(w).alias(f"p90_pct_recarga_adicional_controle_u{months}m"),
            # F.percentile_approx("pct_recarga_adicional_controle", 0.95).over(w).alias(f"p95_pct_recarga_adicional_controle_u{months}m"),
            # F.percentile_approx("pct_recarga_adicional_controle", 0.99).over(w).alias(f"p99_pct_recarga_adicional_controle_u{months}m"),
            # F.percentile_approx("pct_rec_digital", 0.25).over(w).alias(f"p25_pct_rec_digital_u{months}m"),
            # F.percentile_approx("pct_rec_digital", 0.50).over(w).alias(f"p50_pct_rec_digital_u{months}m"),
            # F.percentile_approx("pct_rec_digital", 0.75).over(w).alias(f"p75_pct_rec_digital_u{months}m"),
            # F.percentile_approx("pct_rec_digital", 0.90).over(w).alias(f"p90_pct_rec_digital_u{months}m"),
            # F.percentile_approx("pct_rec_digital", 0.95).over(w).alias(f"p95_pct_rec_digital_u{months}m"),
            # F.percentile_approx("pct_rec_digital", 0.99).over(w).alias(f"p99_pct_rec_digital_u{months}m"),
            # F.percentile_approx("pct_rec_varejo", 0.25).over(w).alias(f"p25_pct_rec_varejo_u{months}m"),
            # F.percentile_approx("pct_rec_varejo", 0.50).over(w).alias(f"p50_pct_rec_varejo_u{months}m"),
            # F.percentile_approx("pct_rec_varejo", 0.75).over(w).alias(f"p75_pct_rec_varejo_u{months}m"),
            # F.percentile_approx("pct_rec_varejo", 0.90).over(w).alias(f"p90_pct_rec_varejo_u{months}m"),
            # F.percentile_approx("pct_rec_varejo", 0.95).over(w).alias(f"p95_pct_rec_varejo_u{months}m"),
            # F.percentile_approx("pct_rec_varejo", 0.99).over(w).alias(f"p99_pct_rec_varejo_u{months}m"),
            # F.percentile_approx("pct_pre_pago", 0.25).over(w).alias(f"p25_pct_pre_pago_u{months}m"),
            # F.percentile_approx("pct_pre_pago", 0.50).over(w).alias(f"p50_pct_pre_pago_u{months}m"),
            # F.percentile_approx("pct_pre_pago", 0.75).over(w).alias(f"p75_pct_pre_pago_u{months}m"),
            # F.percentile_approx("pct_pre_pago", 0.90).over(w).alias(f"p90_pct_pre_pago_u{months}m"),
            # F.percentile_approx("pct_pre_pago", 0.95).over(w).alias(f"p95_pct_pre_pago_u{months}m"),
            # F.percentile_approx("pct_pre_pago", 0.99).over(w).alias(f"p99_pct_pre_pago_u{months}m"),
            # F.percentile_approx("pct_flex", 0.25).over(w).alias(f"p25_pct_flex_u{months}m"),
            # F.percentile_approx("pct_flex", 0.50).over(w).alias(f"p50_pct_flex_u{months}m"),
            # F.percentile_approx("pct_flex", 0.75).over(w).alias(f"p75_pct_flex_u{months}m"),
            # F.percentile_approx("pct_flex", 0.90).over(w).alias(f"p90_pct_flex_u{months}m"),
            # F.percentile_approx("pct_flex", 0.95).over(w).alias(f"p95_pct_flex_u{months}m"),
            # F.percentile_approx("pct_flex", 0.99).over(w).alias(f"p99_pct_flex_u{months}m"),
            # F.percentile_approx("pct_autocontrole", 0.25).over(w).alias(f"p25_pct_autocontrole_u{months}m"),
            # F.percentile_approx("pct_autocontrole", 0.50).over(w).alias(f"p50_pct_autocontrole_u{months}m"),
            # F.percentile_approx("pct_autocontrole", 0.75).over(w).alias(f"p75_pct_autocontrole_u{months}m"),
            # F.percentile_approx("pct_autocontrole", 0.90).over(w).alias(f"p90_pct_autocontrole_u{months}m"),
            # F.percentile_approx("pct_autocontrole", 0.95).over(w).alias(f"p95_pct_autocontrole_u{months}m"),
            # F.percentile_approx("pct_autocontrole", 0.99).over(w).alias(f"p99_pct_autocontrole_u{months}m"),
            # F.percentile_approx("pct_digital", 0.25).over(w).alias(f"p25_pct_digital_u{months}m"),
            # F.percentile_approx("pct_digital", 0.50).over(w).alias(f"p50_pct_digital_u{months}m"),
            # F.percentile_approx("pct_digital", 0.75).over(w).alias(f"p75_pct_digital_u{months}m"),
            # F.percentile_approx("pct_digital", 0.90).over(w).alias(f"p90_pct_digital_u{months}m"),
            # F.percentile_approx("pct_digital", 0.95).over(w).alias(f"p95_pct_digital_u{months}m"),
            # F.percentile_approx("pct_digital", 0.99).over(w).alias(f"p99_pct_digital_u{months}m"),
            # F.percentile_approx("pct_flex_lag", 0.25).over(w).alias(f"p25_pct_flex_lag_u{months}m"),
            # F.percentile_approx("pct_flex_lag", 0.50).over(w).alias(f"p50_pct_flex_lag_u{months}m"),
            # F.percentile_approx("pct_flex_lag", 0.75).over(w).alias(f"p75_pct_flex_lag_u{months}m"),
            # F.percentile_approx("pct_flex_lag", 0.90).over(w).alias(f"p90_pct_flex_lag_u{months}m"),
            # F.percentile_approx("pct_flex_lag", 0.95).over(w).alias(f"p95_pct_flex_lag_u{months}m"),
            # F.percentile_approx("pct_flex_lag", 0.99).over(w).alias(f"p99_pct_flex_lag_u{months}m"),
            # F.percentile_approx("pct_linha_ativa", 0.25).over(w).alias(f"p25_pct_linha_ativa_lag_u{months}m"),
            # F.percentile_approx("pct_linha_ativa", 0.50).over(w).alias(f"p50_pct_linha_ativa_lag_u{months}m"),
            # F.percentile_approx("pct_linha_ativa", 0.75).over(w).alias(f"p75_pct_linha_ativa_lag_u{months}m"),
            # F.percentile_approx("pct_linha_ativa", 0.90).over(w).alias(f"p90_pct_linha_ativa_lag_u{months}m"),
            # F.percentile_approx("pct_linha_ativa", 0.95).over(w).alias(f"p95_pct_linha_ativa_lag_u{months}m"),
            # F.percentile_approx("pct_linha_ativa", 0.99).over(w).alias(f"p99_pct_linha_ativa_lag_u{months}m"),
            # F.percentile_approx("pct_expirado_recente", 0.25).over(w).alias(f"p25_pct_expirado_recente_u{months}m"),
            # F.percentile_approx("pct_expirado_recente", 0.50).over(w).alias(f"p50_pct_expirado_recente_u{months}m"),
            # F.percentile_approx("pct_expirado_recente", 0.75).over(w).alias(f"p75_pct_expirado_recente_u{months}m"),
            # F.percentile_approx("pct_expirado_recente", 0.90).over(w).alias(f"p90_pct_expirado_recente_u{months}m"),
            # F.percentile_approx("pct_expirado_recente", 0.95).over(w).alias(f"p95_pct_expirado_recente_u{months}m"),
            # F.percentile_approx("pct_expirado_recente", 0.99).over(w).alias(f"p99_pct_expirado_recente_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_DIGITAL", 0.25).over(w).alias(f"p25_pct_RECARGA_DIGITAL_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_DIGITAL", 0.50).over(w).alias(f"p50_pct_RECARGA_DIGITAL_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_DIGITAL", 0.75).over(w).alias(f"p75_PCT_RECARGA_DIGITAL_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_DIGITAL", 0.90).over(w).alias(f"p90_PCT_RECARGA_DIGITAL_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_DIGITAL", 0.95).over(w).alias(f"p95_PCT_RECARGA_DIGITAL_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_DIGITAL", 0.99).over(w).alias(f"p99_PCT_RECARGA_DIGITAL_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_VAREJO", 0.25).over(w).alias(f"p25_PCT_RECARGA_VAREJO_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_VAREJO", 0.50).over(w).alias(f"p50_PCT_RECARGA_VAREJO_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_VAREJO", 0.75).over(w).alias(f"p75_PCT_RECARGA_VAREJO_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_VAREJO", 0.90).over(w).alias(f"p90_PCT_RECARGA_VAREJO_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_VAREJO", 0.95).over(w).alias(f"p95_PCT_RECARGA_VAREJO_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_VAREJO", 0.99).over(w).alias(f"p99_PCT_RECARGA_VAREJO_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_TELECOM", 0.25).over(w).alias(f"p25_PCT_RECARGA_TELECOM_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_TELECOM", 0.50).over(w).alias(f"p50_PCT_RECARGA_TELECOM_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_TELECOM", 0.75).over(w).alias(f"p75_PCT_RECARGA_TELECOM_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_TELECOM", 0.90).over(w).alias(f"p90_PCT_RECARGA_TELECOM_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_TELECOM", 0.95).over(w).alias(f"p95_PCT_RECARGA_TELECOM_u{months}m"),
            # F.percentile_approx("PCT_RECARGA_TELECOM", 0.99).over(w).alias(f"p99_PCT_RECARGA_TELECOM_u{months}m"),
            # F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.25).over(w).alias(f"p25_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.50).over(w).alias(f"p50_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.75).over(w).alias(f"p75_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.90).over(w).alias(f"p90_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.95).over(w).alias(f"p95_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_GRUPO_CARTAO_DOMINANTE", 0.99).over(w).alias(f"p99_PCT_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_DSC_GRUPO_CARTAO_DOMINANTE", 0.25).over(w).alias(f"p25_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_DSC_GRUPO_CARTAO_DOMINANTE", 0.50).over(w).alias(f"p50_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_DSC_GRUPO_CARTAO_DOMINANTE", 0.75).over(w).alias(f"p75_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_DSC_GRUPO_CARTAO_DOMINANTE", 0.90).over(w).alias(f"p90_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_DSC_GRUPO_CARTAO_DOMINANTE", 0.95).over(w).alias(f"p95_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),
            # F.percentile_approx("PCT_DSC_GRUPO_CARTAO_DOMINANTE", 0.99).over(w).alias(f"p99_PCT_DSC_GRUPO_CARTAO_DOMINANTE_u{months}m"),


        # =========================
        # PERSISTÊNCIA
        # =========================
         F.sum("flg_uso_sos").over(w).alias(f"qtd_meses_uso_sos_u{months}m"),
         F.sum("flg_quebra_padrao").over(w).alias(f"qtd_meses_quebra_padrao_u{months}m"),
         F.sum("flg_uso_recarga_adicional_controle").over(w).alias(f"qtd_meses_uso_recarga_adicional_controle_u{months}m"),
        # F.sum("flg_canal_diversificado").over(w).alias(f"qtd_meses_canal_diversificado_u{months}m"),
        #  F.sum("flg_bloqueado").over(w).alias(f"qtd_meses_bloqueado_u{months}m"),
        #  F.sum("flg_restricao_credito").over(w).alias(f"qtd_meses_restricao_credito_u{months}m"),
        # F.sum("flg_nao_humano").over(w).alias(f"qtd_meses_nao_humano_u{months}m"),
        # F.sum("flg_tecnico").over(w).alias(f"qtd_meses_tecnico_u{months}m"),
        # F.sum("flg_migracao_plataforma").over(w).alias(f"qtd_meses_migracao_plataforma_u{months}m"),
        #  F.sum("flg_upgrade_flex").over(w).alias(f"qtd_meses_upgrade_flex_u{months}m"),
        #  F.sum("flg_downgrade_flex").over(w).alias(f"qtd_meses_downgrade_flex_u{months}m"),
        # F.sum("flg_excluir_modelo").over(w).alias(f"qtd_meses_excluir_modelo_u{months}m"),
        #  F.sum("flg_expirado_recente").over(w).alias(f"qtd_meses_expirado_recente_u{months}m"),
        #  F.sum("flg_churn_profundo").over(w).alias(f"qtd_meses_churn_profundo_u{months}m"),
        # F.sum("flg_status_ruido").over(w).alias(f"qtd_meses_status_ruido_u{months}m"),
        # F.sum("flg_cliente_elegivel").over(w).alias(f"qtd_meses_cliente_elegivel_u{months}m"),
        # F.sum("FLG_RECARGA_BANCO").over(w).alias(f"qtd_meses_FLG_RECARGA_BANCO_u{months}m"),
        # F.sum("FLG_RECARGA_DIGITAL").over(w).alias(f"qtd_meses_FLG_RECARGA_DIGITAL_u{months}m"),
        # F.sum("FLG_RECARGA_VAREJO").over(w).alias(f"qtd_meses_FLG_RECARGA_VAREJO_u{months}m"),
        # F.sum("FLG_RECARGA_TELECOM").over(w).alias(f"qtd_meses_FLG_RECARGA_TELECOM_u{months}m"),
        # F.sum("FLG_RECARGA_TECNICA").over(w).alias(f"qtd_meses_FLG_RECARGA_TECNICA_u{months}m"),
        # F.sum("FLG_RECARGA_ONLINE").over(w).alias(f"qtd_meses_FLG_RECARGA_ONLINE_u{months}m"),
        # F.sum("FLG_ATIVACAO_PROMO").over(w).alias(f"qtd_meses_FLG_ATIVACAO_PROMO_u{months}m"),
        # F.sum("FLG_PACOTE_SERVICO").over(w).alias(f"qtd_meses_FLG_PACOTE_SERVICO_u{months}m"),
        # F.sum("FLG_CHIP").over(w).alias(f"qtd_meses_FLG_CHIP_u{months}m"),
         F.sum("perfil_emergencial").over(w).alias(f"qtd_meses_perfil_emergencial_u{months}m"),
         F.sum("perfil_incentivado").over(w).alias(f"qtd_meses_perfil_incentivado_u{months}m"),



        # =========================
        # FLAGS TEMPORAIS
        # =========================
         F.max("flg_uso_sos").over(w).alias(f"flg_uso_sos_u{months}m"),
         F.max("flg_quebra_padrao").over(w).alias(f"fx_dependencia_bonus_u{months}m"),
        #  F.max("flg_uso_recarga_adicional_controle").over(w).alias(f"flg_uso_recarga_adicional_controle_u{months}m"),
        # F.max("flg_canal_diversificado").over(w).alias(f"flg_canal_diversificado_u{months}m"),
        #  F.max("flg_bloqueado").over(w).alias(f"flg_bloqueado_u{months}m"),
        #  F.max("flg_restricao_credito").over(w).alias(f"flg_restricao_credito_u{months}m"),
        # F.max("flg_nao_humano").over(w).alias(f"flg_nao_humano_u{months}m"),
        # F.max("flg_tecnico").over(w).alias(f"flg_tecnico_u{months}m"),
        # F.max("flg_migracao_plataforma").over(w).alias(f"flg_migracao_plataforma_u{months}m"),
        #  F.max("flg_upgrade_flex").over(w).alias(f"flg_upgrade_flex_u{months}m"),
        #  F.max("flg_downgrade_flex").over(w).alias(f"flg_downgrade_flex_u{months}m"),
        # F.max("flg_excluir_modelo").over(w).alias(f"flg_excluir_modelo_u{months}m"),
        #  F.max("flg_expirado_recente").over(w).alias(f"flg_expirado_recente_u{months}m"),
        #  F.max("flg_churn_profundo").over(w).alias(f"flg_churn_profundo_u{months}m"),
        # F.max("flg_status_ruido").over(w).alias(f"flg_status_ruido_u{months}m"),
        # F.max("flg_cliente_elegivel").over(w).alias(f"flg_cliente_elegivel_u{months}m"),
        # F.max("FLG_RECARGA_BANCO").over(w).alias(f"FLG_RECARGA_BANCO_u{months}m"),
        # F.max("FLG_RECARGA_DIGITAL").over(w).alias(f"FLG_RECARGA_DIGITAL_u{months}m"),
        # F.max("FLG_RECARGA_VAREJO").over(w).alias(f"FLG_RECARGA_VAREJO_u{months}m"),
        # F.max("FLG_RECARGA_TELECOM").over(w).alias(f"FLG_RECARGA_TELECOM_u{months}m"),
        # F.max("FLG_RECARGA_TECNICA").over(w).alias(f"FLG_RECARGA_TECNICA_u{months}m"),
        # F.max("FLG_RECARGA_ONLINE").over(w).alias(f"FLG_RECARGA_ONLINE_u{months}m"),
        # F.max("FLG_ATIVACAO_PROMO").over(w).alias(f"FLG_ATIVACAO_PROMO_u{months}m"),
        # F.max("FLG_PACOTE_SERVICO").over(w).alias(f"FLG_PACOTE_SERVICO_u{months}m"),
        # F.max("FLG_CHIP").over(w).alias(f"FLG_CHIP_u{months}m"),
         F.max("perfil_emergencial").over(w).alias(f"perfil_emergencial_u{months}m"),
         F.max("perfil_incentivado").over(w).alias(f"perfil_incentivado_u{months}m"),


        # =========================
        # RANKING
        # =========================

        # =========================
        # FEATURES CRUZADAS
        # =========================

        # =========================
        # VARIAVEIS DE COMPARAÇÃO
        # Comparamos janelas curtas com janelas longas para identificar mudanças de comportamento.
        # =========================

        # =========================
        #       DETERIORAÇÃO/MELHORA
        # =========================


        # =========================
        #       ACELERAÇÃO
        #       Exemplo: Se a razão for > 1, o cliente está gastando mais do que o seu padrão histórico.
        # =========================


        # =========================
        #       DESACELERAÇÃO
        #       Exemplo: Se a razão for < 1, o cliente está gastando menos do que o seu padrão histórico.
        # =========================



    )

    return df_w


## Construção de janela de 1 mês do datraframe df_book

In [323]:
df_m1 = window_features(df_book, 1)

In [324]:
df_m1.printSchema()

root
 |-- NUM_CPF: string (nullable = true)
 |-- SAFRA_RECARGA: string (nullable = true)
 |-- mean_pct_rec_sos_u1m: double (nullable = true)
 |-- mean_pct_rec_com_bonus_u1m: double (nullable = true)
 |-- mean_gap_medio_recarga_u1m: double (nullable = true)
 |-- mean_recencia_dias_u1m: double (nullable = true)
 |-- qtd_recargas_u1m: long (nullable = false)
 |-- qtd_rec_sos_u1m: long (nullable = false)
 |-- qtd_canais_distintos_u1m: long (nullable = false)
 |-- qtd_eventos_u1m: long (nullable = false)
 |-- std_vlr_medio_recarga_u1m: double (nullable = true)
 |-- std_pct_rec_com_bonus_u1m: double (nullable = true)
 |-- std_gap_medio_recarga_u1m: double (nullable = true)
 |-- std_recencia_dias_u1m: double (nullable = true)
 |-- sum_qtd_recargas_u1m: long (nullable = true)
 |-- sum_vlr_recarga_u1m: double (nullable = true)
 |-- sum_vlr_total_bonus_u1m: double (nullable = true)
 |-- sum_vlr_total_real_u1m: double (nullable = true)
 |-- sum_qtd_rec_sos_u1m: double (nullable = true)
 |-- sum_q

In [325]:
# numero de colunas do dataframe pyspark
len(df_m1.columns)


30

## Construção de janela de 3 meses do datraframe df_book

In [326]:
df_m3 = window_features(df_book, 3)

## Construção de janela de 6 meses do datraframe df_book

In [327]:
df_m6 = window_features(df_book, 6)

## Construção de janela de 9 meses do datraframe df_book

---



In [328]:
df_m9 = window_features(df_book, 9)

## Construção de janela de 12 meses do datraframe df_book



---



In [329]:
df_m12 = window_features(df_book, 12)

## Book final com as janelas

In [330]:
df_features = df_book \
    .select("*") \
    .join(df_m1, ["NUM_CPF", "SAFRA_RECARGA"], "left") \
    .join(df_m3, ["NUM_CPF", "SAFRA_RECARGA"], "left") \
    .join(df_m6, ["NUM_CPF", "SAFRA_RECARGA"], "left") \
    .join(df_m9, ["NUM_CPF", "SAFRA_RECARGA"], "left") \
    .join(df_m12, ["NUM_CPF", "SAFRA_RECARGA"], "left")

In [331]:
df_features.printSchema()

root
 |-- NUM_CPF: string (nullable = true)
 |-- SAFRA_RECARGA: string (nullable = true)
 |-- qtd_recargas: long (nullable = false)
 |-- vlr_total_recarga: double (nullable = true)
 |-- vlr_medio_recarga: double (nullable = true)
 |-- vlr_max_recarga: double (nullable = true)
 |-- vlr_min_recarga: double (nullable = true)
 |-- qtd_telefones: long (nullable = false)
 |-- dt_primeira_recarga: date (nullable = true)
 |-- dt_ultima_recarga: date (nullable = true)
 |-- vlr_total_bonus: double (nullable = true)
 |-- vlr_medio_bonus: double (nullable = true)
 |-- vlr_total_real: double (nullable = true)
 |-- vlr_medio_real: double (nullable = true)
 |-- qtd_rec_madrugada: long (nullable = true)
 |-- qtd_rec_manha: long (nullable = true)
 |-- qtd_rec_tarde: long (nullable = true)
 |-- qtd_rec_noite: long (nullable = true)
 |-- fx_hora_dominante: string (nullable = true)
 |-- qtd_rec_sos: double (nullable = true)
 |-- vlr_medio_rec_sos: double (nullable = true)
 |-- pct_rec_sos: double (nullabl

In [332]:
len (df_features.columns)

254

In [333]:
df_book_202501 = df_features.filter(F.col('SAFRA_RECARGA') == 202501)
df_book_202412 = df_features.filter(F.col('SAFRA_RECARGA') == 202412)
df_book_202411 = df_features.filter(F.col('SAFRA_RECARGA') == 202411)
df_book_202410 = df_features.filter(F.col('SAFRA_RECARGA') == 202410)

In [334]:
# df_book_202411.show(5)

In [335]:
#df_book.filter(F.col('NUM_CPF') == '79T9YTYZTZZ').show()
#df_features.filter( F.col('SAFRA_RECARGA') >= 202310).show()

In [336]:
# Gerer o arquivo parquet do book por safra 202501
# import os
# from pathlib import Path
# import pyarrow as pa
# import pyarrow.parquet as pq


# caminho_saida = f'/content/drive/MyDrive/HACKATHON_2025/data/book'

# caminho_completo = os.path.join(caminho_saida, f'features_recarga.parquet')

# '''df_book_202501.write \
#   .format("parquet") \
#   .mode("overwrite") \
#   .option("compression", "snappy") \
#   .option("parquet.block.size", 134217728) \
#   .partitionBy("SAFRA_RECARGA") \
#   .save(caminho_completo)'''

In [337]:
# Gerer o arquivo parquet do book por safra 202412
# import os
# from pathlib import Path
# import pyarrow as pa
# import pyarrow.parquet as pq


# caminho_saida = f'/content/drive/MyDrive/HACKATHON_2025/data/book'

# caminho_completo = os.path.join(caminho_saida, f'features_recarga.parquet')

# df_book_202412.write \
#   .format("parquet") \
#   .mode("overwrite") \
#   .option("compression", "snappy") \
#   .option("parquet.block.size", 134217728) \
#   .partitionBy("SAFRA_RECARGA") \
#   .save(caminho_completo)

In [338]:
# spark.conf.set("spark.sql.parquet.compression.codec", "snappy")
# spark.conf.set("spark.sql.parquet.mergeSchema", "false")
# spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

In [345]:
from google.colab import drive
drive.mount("/content/drive")
from pyspark.sql import functions as F
from pyspark.sql.types import NullType
import math

def save_parquet_wide_df(
    df,
    base_path,
    block_size=300,
    n_partitions=24,
    mode="overwrite"
):
    """
    Salva DataFrame Spark muito largo (ex: 3000+ colunas)
    em blocos de colunas no Google Drive (ou FS estável).

    Cada bloco é um parquet separado.
    """

    # =========================
    # 1. Sanear schema
    # =========================
    for field in df.schema.fields:
        if isinstance(field.dataType, NullType):
            df = df.withColumn(field.name, F.lit(None).cast("string"))

    # =========================
    # 2. Repartition (NUNCA coalesce)
    # =========================
    df = df.repartition(n_partitions)

    # =========================
    # 3. Preparar blocos
    # =========================
    cols = df.columns
    total_cols = len(cols)
    n_blocks = math.ceil(total_cols / block_size)

    print(f"→ Total colunas: {total_cols}")
    print(f"→ Blocos: {n_blocks} (≈ {block_size} colunas/bloco)")
    print(f"→ Partições: {n_partitions}")

    # =========================
    # 4. Salvar metadados
    # =========================
    spark.createDataFrame(
        [(i, c) for i, c in enumerate(cols)],
        ["ordem", "coluna"]
    ).write.mode(mode).parquet(f"{base_path}/metadata")

    # =========================
    # 5. Salvar blocos
    # =========================
    for i in range(n_blocks):
        start = i * block_size
        end = min(start + block_size, total_cols)
        block_cols = cols[start:end]

        block_path = f"{base_path}/block{i:03d}"

        print(f"→ Salvando bloco {i+1}/{n_blocks} | cols {start}:{end}")

        (
            df.select(*block_cols)
              .write
              .mode(mode)
              .parquet(block_path)
        )

    print("✅ Salvamento concluído com sucesso.")



save_parquet_wide_df(
    df=df_book_202501,
    base_path="/content/drive/MyDrive/HACKATHON_2025/recargas_book_202501",
    block_size=300,     # ajuste se necessário (300–500)
    n_partitions=24     # ajuste c
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
→ Total colunas: 254
→ Blocos: 1 (≈ 300 colunas/bloco)
→ Partições: 24
→ Salvando bloco 1/1 | cols 0:254
✅ Salvamento concluído com sucesso.


In [ ]:
# Gerer o arquivo parquet do book por safra 202411
# import os
# from pathlib import Path
# import pyarrow as pa
# import pyarrow.parquet as pq


# caminho_saida = f'/content/drive/MyDrive/HACKATHON_2025/data/book'

# caminho_completo = os.path.join(caminho_saida, f'features_recarga.parquet')

# df_book_202411.write \
#   .format("parquet") \
#   .mode("overwrite") \
#   .option("compression", "snappy") \
#   .option("parquet.block.size", 134217728) \
#   .partitionBy("SAFRA_RECARGA") \
#   .save(caminho_completo)

In [ ]:
# Gerer o arquivo parquet do book por safra 202410
# import os
# from pathlib import Path
# import pyarrow as pa
# import pyarrow.parquet as pq


# caminho_saida = f'/content/drive/MyDrive/HACKATHON_2025/data/book'

# caminho_completo = os.path.join(caminho_saida, f'features_recarga.parquet')

# '''df_book_202410.write \
#   .format("parquet") \
#   .mode("overwrite") \
#   .option("compression", "snappy") \
#   .option("parquet.block.size", 134217728) \
#   .partitionBy("SAFRA_RECARGA") \
#   .save(caminho_completo)'''